Individual_Z1~4
12metaanalysis
bandpass
VAE

In [1]:
import scipy.io
import os
import h5py
import numpy as np
import os
import torchvision
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from tqdm import tqdm
import time
import torch
import torch.nn as nn
from torch import optim
import sys
from torch.utils.data import TensorDataset ,DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from torch.nn import Sigmoid

# -*- coding: utf-8 -*-
from joblib import Parallel, delayed
from time import time
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import torch.nn.init
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import cross_val_score
import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
import requests
from torch.nn import Softmax
import matplotlib.pyplot as plt
from torch.nn import LogSoftmax
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from torchvision import datasets, transforms

import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from tqdm import tqdm
import time
import torch
import torch.nn as nn
from torch import optim

from torch.utils.data import TensorDataset ,DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from torch.nn import Sigmoid
from torch.optim.lr_scheduler import ReduceLROnPlateau
import sklearn

import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import torch.nn.init
from sklearn.model_selection import cross_val_predict 
from sklearn.model_selection import cross_val_score
import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR
import requests
from torch.nn import Softmax
import matplotlib.pyplot as plt
from torch.nn import LogSoftmax
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import gc 
import torch.nn.functional as F
import random

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 2019
seed_everything(SEED)

# パスの定義

In [3]:
DATA_PATH = "/mnt/koyama/koyama/data_old/langMSMALL_csv/complete/"

In [4]:
METADATA_PATH = "/mnt/koyama/koyama/data_old/NST_csv/"

In [5]:
METADATA_PATH2 = "/mnt/koyama/koyama/data_old/NQT_csv/"

In [6]:
BANDDATA_PATH = '/mnt/koyama/koyama/data_old/langBandPassedcsv/0.01-0.1Hz/complete/'

# データロード
##  ファイル名とmetanalysisデータ

### NQR

In [10]:
hand = pd.read_csv(METADATA_PATH2+"not_normalize_hand_NQT.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)
calculation = pd.read_csv(METADATA_PATH2+"not_normalize_calculation_NQT.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)
listening = pd.read_csv(METADATA_PATH2+"not_normalize_listening_NQT.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)
mentalizing = pd.read_csv(METADATA_PATH2+"not_normalize_mentalizing_NQT.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)

-------------------------------------------------------

### NST

In [8]:
hand = pd.read_csv("/mnt/koyama/koyama/data_old/NST_csv/not_normalize_hand_NST.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)
calculation = pd.read_csv("/mnt/koyama/koyama/data_old/NST_csv/not_normalize_calculation_NST.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)
listening = pd.read_csv("/mnt/koyama/koyama/data_old/NST_csv/not_normalize_listening_NST.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)
mentalizing = pd.read_csv("/mnt/koyama/koyama/data_old/NST_csv/not_normalize_mentalizing_NST.csv"
            ,sep='\t',na_values='                      NaN',header=None)[0].fillna(np.nan).astype(np.float32)

-------------------------------------------------------

In [11]:
metanalysis = list()
metanalysis.extend([hand[~hand.isnull()],
calculation[~calculation.isnull()],
listening[~listening.isnull()],
mentalizing[~mentalizing.isnull()]])
metanalysis = np.vstack(metanalysis)
metanalysis = torch.tensor(metanalysis).to("cuda")

### 12個の場合

In [12]:
calculation = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/calculation.csv",header=None).astype(np.float32).iloc[:,0]
default = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/default.csv",header=None).astype(np.float32).iloc[:,0]
emotional = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/emotional.csv",header=None).astype(np.float32).iloc[:,0]
language = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/language.csv",header=None).astype(np.float32).iloc[:,0]
listening = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/listening.csv",header=None).astype(np.float32).iloc[:,0]
mentalizing = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/mentalizing.csv",header=None).astype(np.float32).iloc[:,0]
motor = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/motor.csv",header=None).astype(np.float32).iloc[:,0]
reading = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/reading.csv",header=None).astype(np.float32).iloc[:,0]
somatosensory = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/somatosensory.csv",header=None).astype(np.float32).iloc[:,0]
task = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/task.csv",header=None).astype(np.float32).iloc[:,0]
visual = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/visual.csv",header=None).astype(np.float32).iloc[:,0]
working_memory = pd.read_csv("/mnt/koyama/koyama/data_old/NQT_csv/new/working_memory.csv",header=None).astype(np.float32).iloc[:,0]

In [13]:
metanalysis = list()
metanalysis.extend([calculation,
default,
emotional,
language,
listening,mentalizing,motor,reading,somatosensory,task,visual,working_memory])
metanalysis = np.vstack(metanalysis)
metanalysis = torch.tensor(metanalysis).to("cuda")

-------------------------------------------------------

In [20]:
sublect_list = os.listdir("/mnt/koyama/koyama/data_old/langMSMALL_csv/complete/")

In [21]:
use_subject_num = pd.read_csv(DATA_PATH+"rest_run1234_random515_1.csv",sep='\t',header=None).iloc[0,3:34]
group1_sub = pd.read_csv(DATA_PATH+"rest_run1234_random515_1.csv",sep='\t',header=None).values

group2_sub = pd.read_csv(DATA_PATH+"rest_run1234_random515_2.csv",sep='\t',header=None).values
group1_sub = pd.read_csv(DATA_PATH+"rest_run1234_random515_1.csv",sep='\t',header=None).values

group2_sub = pd.read_csv(DATA_PATH+"rest_run1234_random515_2.csv",sep='\t',header=None).values

group1_sub = pd.DataFrame(sublect_list)[0][:2086].str[:6].isin(group1_sub[0].astype(str))
group2_sub = pd.DataFrame(sublect_list)[0][:2086].str[:6].isin(group2_sub[0].astype(str))
sublect_list = pd.DataFrame(sublect_list)[0][:2086]
group1_sub_path = sublect_list[group1_sub]
group2_sub_path = sublect_list[group2_sub]

In [63]:
import psutil 

# メモリ容量を取得
mem = psutil.virtual_memory() 
print(mem.total)

#[結果] 8492281856

# メモリ使用量を取得 
print(mem.used/1024**3)
#[結果] 4748627968

# メモリ空き容量を取得 
print(mem.available/1024**3)
#[結果] 3743653888

269961125888
44.24778366088867
204.79476928710938


##  PYTORCH DATASET定義
PyTorch transforms/Dataset/DataLoaderの基本動作を確認する
https://qiita.com/takurooo/items/e4c91c5d78059f92e76d

In [17]:
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data,label=None, transform=None):
        self.transform = transform
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out_data = self.data[idx]
        #out_label =  self.label[idx]

        if self.transform:
            out_data = self.transform(out_data)

        return out_data

# AUTOENCODER

In [54]:
class MixedApproachTiedAutoEncoder(nn.Module):
    def __init__(self, inp, out,bias=False):
        super().__init__()
        seed_everything(SEED)#seed_everythingをあることでmodelの初期値が固定されるはず
        self.encoder = nn.Linear(inp, out, bias=bias)
        #self.bias = nn.Parameter(torch.rand(59412)*0.001)
    def forward(self, input):
        encoded_feats = self.encoder(input)
        reconstructed_output = F.linear(encoded_feats, self.encoder.weight.t())
        return encoded_feats, reconstructed_output

In [77]:
class MixedApproachTiedAutoEncoder(nn.Module):
    def __init__(self, inp, out,bias=False):
        super().__init__()
        seed_everything(SEED)#seed_everythingをあることでmodelの初期値が固定されるはず
        self.encoder = nn.Linear(inp, out, bias=bias)
        self.prelu1 = nn.PReLU()
        self.prelu2 = nn.PReLU()
        #self.bias = nn.Parameter(torch.rand(59412)*0.001)
    def forward(self, input):
        encoded_feats = self.prelu1(self.encoder(input))
        reconstructed_output = self.prelu2(F.linear(encoded_feats, self.encoder.weight.t()))
        return encoded_feats, reconstructed_output

In [21]:
class AutoEncoder(nn.Module):
    def __init__(self, inp, out,bias=False):
        super().__init__()
        seed_everything(SEED)#seed_everythingをあることでmodelの初期値が固定されるはず
        self.encoder = nn.Linear(inp, out, bias=bias)
        self.decoder = nn.Linear(out, inp, bias=bias)
        
    def forward(self, input):
        encoded_feats = self.encoder(input)
        reconstructed_output = self.decoder(encoded_feats)
        return encoded_feats, reconstructed_output

In [19]:
class MixedApproachTiedMultiAutoEncoder(nn.Module):
    def __init__(self, inp,xx,out,bias=False):
        super().__init__()
        self.encoder = nn.Linear(inp, xx, bias=bias)
        self.encoder2 = nn.Linear(xx, out, bias=bias)
        self.prelu = nn.PReLU()
        self.prelu2 = nn.PReLU()
        self.biasxx = nn.Parameter(torch.rand(xx)*0.001)
        self.bias = nn.Parameter(torch.rand(inp)*0.001)
    def forward(self, input):
        encoded_feats = self.prelu(self.encoder(input))
        encoded_feats = self.encoder2(encoded_feats)
        reconstructed_output = self.prelu2(F.linear(encoded_feats, self.encoder2.weight.t()) + self.biasxx)
        reconstructed_output = F.linear(reconstructed_output, self.encoder.weight.t()) + self.bias
        
        return encoded_feats, reconstructed_output

In [20]:
MixedApproachTiedMultiAutoEncoder(59412,512,5,bias=True)

MixedApproachTiedMultiAutoEncoder(
  (encoder): Linear(in_features=59412, out_features=512, bias=True)
  (encoder2): Linear(in_features=512, out_features=5, bias=True)
  (prelu): PReLU(num_parameters=1)
  (prelu2): PReLU(num_parameters=1)
)

In [18]:
class MixedApproachTiedMultiAutoEncoder(nn.Module):
    def __init__(self, inp,xx,out,bias=False):
        super().__init__()
        self.encoder = nn.Linear(inp, xx, bias=bias)
        self.encoder2 = nn.Linear(xx, out, bias=bias)
        self.prelu = nn.PReLU()
        self.prelu2 = nn.PReLU()
    def forward(self, input):
        encoded_feats = self.prelu(self.encoder(input))
        encoded_feats = self.encoder2(encoded_feats)
        reconstructed_output = self.prelu2(F.linear(encoded_feats, self.encoder2.weight.t()))
        reconstructed_output = F.linear(reconstructed_output, self.encoder.weight.t())
        
        return encoded_feats, reconstructed_output

In [64]:
def loss_fuc(x,output,weight,metanalysis,alpha,deco_weight=None):
    """weight 59412x4"""
    num = len(metanalysis)
    weightdata = weight[:num,:]
    meta_std = torch.std(metanalysis,dim=1)
    weight_std = torch.std(weightdata,dim=1)
    meta_mean = torch.mean(metanalysis,dim=1)[:,None]
    weight_mean = torch.mean(weightdata,dim=1)[:,None]
    covar = torch.mean((weightdata - weight_mean)*torch.tensor((metanalysis-meta_mean)),dim=1)
    corr = covar/(weight_std*meta_std)
    
    corr_mean = torch.mean(corr)
    rloss = 1-corr_mean
    
    
    mse = mseloss(x,output)
    
    if deco_weight is not None:
        weightdata = deco_weight.t()[:num,:]
        weight_std = torch.std(weightdata,dim=1)
        weight_mean = torch.mean(weightdata,dim=1)[:,None]
        covar = torch.mean((weightdata - weight_mean)*torch.tensor((metanalysis-meta_mean)),dim=1)
        corr = covar/(weight_std*meta_std)
        corr_mean_de= torch.mean(corr)
        rloss_de = 1-corr_mean
        return mse + alpha*rloss + alpha*rloss_de,corr_mean ,mse,corr_mean_de
        
    return mse + alpha*rloss,corr_mean ,mse

In [65]:
def sparse_loss(autoencoder, images):
    loss = 0
    values = images

    h = autoencoder.encoder(images)
    
    loss += torch.mean(torch.abs(h))
    return loss

In [21]:
model = AutoEncoder(59412,5,bias=True)

NameError: name 'AutoEncoder' is not defined

In [32]:
model.decoder.weight.shape

torch.Size([59412, 5])

In [33]:
model.encoder.weight.shape

torch.Size([5, 59412])

# 前処理の方法

### indvidual1
#### グループでは全体で標準化、学習も全体で

In [89]:
sample = pd.read_csv(DATA_PATH+group1_sub_path.iloc[0],sep='\t',header=None)
sample = sample.values[None,:,:]
for subnum in tqdm_notebook(np.sort(group1_sub_path.values[1:40])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    sample = np.append(sample,A.values[None,:,:],axis=0)

test_sample = pd.read_csv(DATA_PATH+group2_sub_path.iloc[0],sep='\t',header=None)
test_sample = test_sample.values[None,:,:]
for subnum in tqdm_notebook(np.sort(group2_sub_path.values[1:80])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    test_sample = np.append(test_sample,A.values[None,:,:],axis=0)



sample = sample[:,:,:-1]
sample = np.reshape(sample.transpose(0,2,1),(40*316,-1))
test_sample = test_sample[:,:,:-1]
test_sample = np.reshape(test_sample.transpose(0,2,1),(80*316,-1))
scaler = StandardScaler()
scaler.fit(np.append(sample,test_sample,axis=0))
sample = scaler.transform(sample)
test_sample = scaler.transform(test_sample)



A Jupyter Widget

A Jupyter Widget

In [5]:
def aa():
    print(1)

In [6]:
X = {"A":{"S":aa}}

In [10]:
ss = X["A"]["S"]
ss()

1


### bandpass の場合

In [ ]:
group1_bandsub_path = group1_sub_path.str.replace("MSMALL","BandPassed")
group2_bandsub_path = group2_sub_path.str.replace("MSMALL","BandPassed")

train_bandsample = pd.read_csv(BANDDATA_PATH+group1_bandsub_path.iloc[0],sep='\t',header=None)
train_bandsample = train_bandsample.values[None,:,:]

for subnum in tqdm_notebook(np.sort(group1_bandsub_path.values[1:40])):
    
    A = pd.read_csv(BANDDATA_PATH+subnum,sep='\t',header=None)
    train_bandsample = np.append(train_bandsample,A.values[None,:,:],axis=0)
    
test_bandsample = pd.read_csv(BANDDATA_PATH+group2_bandsub_path.iloc[0],sep='\t',header=None)
test_bandsample = test_bandsample.values[None,:,:]

for subnum in tqdm_notebook(np.sort(group2_bandsub_path.values[1:80])):
    
    A = pd.read_csv(BANDDATA_PATH+subnum,sep='\t',header=None)
    test_bandsample = np.append(test_bandsample,A.values[None,:,:],axis=0)
    
    
test_bandsample = test_bandsample[:,:,:-1]
train_bandsample = train_bandsample[:,:,:-1]
train_bandsample = np.reshape(train_bandsample.transpose(0,2,1),(40*316,-1))
test_bandsample = np.reshape(test_bandsample.transpose(0,2,1),(80*316,-1))
scaler = StandardScaler()
scaler.fit(np.append(train_bandsample,test_bandsample,axis=0))
train_bandsample = scaler.transform(train_bandsample)
test_bandsample = scaler.transform(test_bandsample)

### indvidual2,
#### グループでは個人で標準化、学習は全体で

In [23]:
group1_sub_path_run1 = group1_sub_path[group1_sub_path.str.contains("run1")] #run1のファイル名パス
group1_sub_path_run2 = group1_sub_path[group1_sub_path.str.contains("run2")]

sample = pd.read_csv(DATA_PATH+group1_sub_path_run1.iloc[0],sep='\t',header=None)
sample = sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(sample)
sample = scaler.transform(sample)
sample = sample[None,:,:]
for subnum in tqdm_notebook(np.sort(group1_sub_path_run1.values[1:40])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    sample = np.append(sample,A,axis=0)


test_sample = pd.read_csv(DATA_PATH+group1_sub_path_run2.iloc[0],sep='\t',header=None)
test_sample = test_sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(test_sample)
test_sample = scaler.transform(test_sample)
test_sample = test_sample[None,:,:]

for subnum in tqdm_notebook(np.sort(group1_sub_path_run2.values[1:80])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    test_sample = np.append(test_sample,A,axis=0)

### bandpass の場合

In [22]:
group1_bandsub_path = group1_sub_path.str.replace("MSMALL","BandPassed")

group1_sub_path_run1 = group1_bandsub_path[group1_bandsub_path.str.contains("run1")] #run1のファイル名パス
group1_sub_path_run2 = group1_bandsub_path[group1_bandsub_path.str.contains("run2")]

sample = pd.read_csv(BANDDATA_PATH+group1_sub_path_run1.iloc[0],sep='\t',header=None)
sample = sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(sample)
sample = scaler.transform(sample)
sample = sample[None,:,:]
for subnum in tqdm_notebook(np.sort(group1_sub_path_run1.values[1:40])):
    
    A = pd.read_csv(BANDDATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    sample = np.append(sample,A,axis=0)


test_sample = pd.read_csv(BANDDATA_PATH+group1_sub_path_run2.iloc[0],sep='\t',header=None)
test_sample = test_sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(test_sample)
test_sample = scaler.transform(test_sample)
test_sample = test_sample[None,:,:]

for subnum in tqdm_notebook(np.sort(group1_sub_path_run2.values[1:80])):
    
    A = pd.read_csv(BANDDATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    test_sample = np.append(test_sample,A,axis=0)

In [24]:
sample = np.reshape(sample,(40*316,-1))
test_sample = np.reshape(test_sample,(80*316,-1))

##### sample:run1
##### test_sample:run2 
##### run1とrun2を混ぜて学習する場合はsample.append(test_sample)

In [53]:
train_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)

test_dataset = MyDataset(torch.tensor(test_sample,dtype=torch.float32))
test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)

#### グループでのweightをつくるための学習コード

In [66]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,13,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        spa_loss = sparse_loss(model,data)
        loss = loss + 0.0001*spa_loss
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:1.0059,mseloss:0.9107
test_epoch [1/20], loss:0.9010,corr:0.9971,mseloss:0.8967
epoch [2/20], sumloss:0.9007,mseloss:0.8948
test_epoch [2/20], loss:0.8971,corr:0.9968,mseloss:0.8923
epoch [3/20], sumloss:0.8976,mseloss:0.8919
test_epoch [3/20], loss:0.8953,corr:0.9972,mseloss:0.8910
epoch [4/20], sumloss:0.8966,mseloss:0.8912
test_epoch [4/20], loss:0.8946,corr:0.9977,mseloss:0.8911
epoch [5/20], sumloss:0.8963,mseloss:0.8910
test_epoch [5/20], loss:0.8945,corr:0.9974,mseloss:0.8906
epoch [6/20], sumloss:0.8960,mseloss:0.8908
test_epoch [6/20], loss:0.8944,corr:0.9977,mseloss:0.8910
epoch [7/20], sumloss:0.8961,mseloss:0.8910
test_epoch [7/20], loss:0.8945,corr:0.9975,mseloss:0.8907
epoch [8/20], sumloss:0.8961,mseloss:0.8909
test_epoch [8/20], loss:0.8946,corr:0.9975,mseloss:0.8909
epoch [9/20], sumloss:0.8959,mseloss:0.8908
test_epoch [9/20], loss:0.8941,corr:0.9977,mseloss:0.8906
epoch [10/20], sumloss:0.8957,mseloss:0.8907
test_epoch [10/20], loss:0.8939,corr:

In [69]:
torch.save(model.state_dict(),"/mnt/cifs/result/NQ_new/bandpass/init_weight/model_initweightrun1.pth")

In [41]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,13,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:0.9317,mseloss:0.9263
test_epoch [1/20], loss:0.9283,corr:0.5409,mseloss:0.9237
epoch [2/20], sumloss:0.9072,mseloss:0.9023
test_epoch [2/20], loss:0.9263,corr:0.4848,mseloss:0.9211
epoch [3/20], sumloss:0.9043,mseloss:0.8990
test_epoch [3/20], loss:0.9261,corr:0.4748,mseloss:0.9209
epoch [4/20], sumloss:0.9038,mseloss:0.8985
test_epoch [4/20], loss:0.9260,corr:0.4744,mseloss:0.9208
epoch [5/20], sumloss:0.9035,mseloss:0.8983
test_epoch [5/20], loss:0.9261,corr:0.4784,mseloss:0.9209
epoch [6/20], sumloss:0.9033,mseloss:0.8981
test_epoch [6/20], loss:0.9261,corr:0.4814,mseloss:0.9209
epoch [7/20], sumloss:0.9032,mseloss:0.8981
test_epoch [7/20], loss:0.9261,corr:0.4872,mseloss:0.9209
epoch [8/20], sumloss:0.9032,mseloss:0.8981
test_epoch [8/20], loss:0.9259,corr:0.4906,mseloss:0.9208
epoch [9/20], sumloss:0.9029,mseloss:0.8978
test_epoch [9/20], loss:0.9259,corr:0.4941,mseloss:0.9208
epoch [10/20], sumloss:0.9030,mseloss:0.8979
test_epoch [10/20], loss:0.9256,corr:

In [42]:
torch.save(model.state_dict(),"/mnt/cifs/result/MS_new13/model_initweight.pth")

In [44]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,10,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:0.9896,mseloss:0.8953
test_epoch [1/20], loss:0.8768,corr:0.9996,mseloss:0.8762
epoch [2/20], sumloss:0.8676,mseloss:0.8670
test_epoch [2/20], loss:0.8746,corr:0.9996,mseloss:0.8739
epoch [3/20], sumloss:0.8660,mseloss:0.8653
test_epoch [3/20], loss:0.8733,corr:0.9995,mseloss:0.8726
epoch [4/20], sumloss:0.8650,mseloss:0.8643
test_epoch [4/20], loss:0.8721,corr:0.9995,mseloss:0.8713
epoch [5/20], sumloss:0.8640,mseloss:0.8632
test_epoch [5/20], loss:0.8715,corr:0.9995,mseloss:0.8707
epoch [6/20], sumloss:0.8637,mseloss:0.8629
test_epoch [6/20], loss:0.8711,corr:0.9995,mseloss:0.8704
epoch [7/20], sumloss:0.8636,mseloss:0.8628
test_epoch [7/20], loss:0.8712,corr:0.9995,mseloss:0.8704
epoch [8/20], sumloss:0.8636,mseloss:0.8628
test_epoch [8/20], loss:0.8712,corr:0.9994,mseloss:0.8704
epoch [9/20], sumloss:0.8635,mseloss:0.8627
test_epoch [9/20], loss:0.8711,corr:0.9995,mseloss:0.8703
epoch [10/20], sumloss:0.8634,mseloss:0.8626
test_epoch [10/20], loss:0.8711,corr:

In [45]:
torch.save(model.state_dict(),"/mnt/cifs/result/Individual_Z3/bandpass/AEnode10/init_weight/model_initweightrun1.pth")

In [71]:
train_dataset = MyDataset(torch.tensor(test_sample[:len(sample)],dtype=torch.float32))
train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)

test_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)

In [47]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,10,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:0.9976,mseloss:0.9033
test_epoch [1/20], loss:0.8770,corr:0.9996,mseloss:0.8765
epoch [2/20], sumloss:0.8741,mseloss:0.8735
test_epoch [2/20], loss:0.8753,corr:0.9996,mseloss:0.8747
epoch [3/20], sumloss:0.8733,mseloss:0.8726
test_epoch [3/20], loss:0.8750,corr:0.9995,mseloss:0.8743
epoch [4/20], sumloss:0.8732,mseloss:0.8725
test_epoch [4/20], loss:0.8738,corr:0.9994,mseloss:0.8730
epoch [5/20], sumloss:0.8717,mseloss:0.8710
test_epoch [5/20], loss:0.8730,corr:0.9995,mseloss:0.8722
epoch [6/20], sumloss:0.8718,mseloss:0.8710
test_epoch [6/20], loss:0.8730,corr:0.9994,mseloss:0.8722
epoch [7/20], sumloss:0.8714,mseloss:0.8706
test_epoch [7/20], loss:0.8730,corr:0.9994,mseloss:0.8722
epoch [8/20], sumloss:0.8714,mseloss:0.8707
test_epoch [8/20], loss:0.8730,corr:0.9995,mseloss:0.8722
epoch [9/20], sumloss:0.8715,mseloss:0.8706
test_epoch [9/20], loss:0.8734,corr:0.9994,mseloss:0.8725
epoch [10/20], sumloss:0.8711,mseloss:0.8702
test_epoch [10/20], loss:0.8732,corr:

In [49]:
torch.save(model.state_dict(),"/mnt/cifs/result/Individual_Z3/bandpass/AEnode10/init_weight/model_initweightrun2.pth")

In [72]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,13,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        spa_loss = sparse_loss(model,data)
        loss = loss + 0.0001*spa_loss
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:1.0098,mseloss:0.9153
test_epoch [1/20], loss:0.9053,corr:0.9972,mseloss:0.9010
epoch [2/20], sumloss:0.9065,mseloss:0.9011
test_epoch [2/20], loss:0.9008,corr:0.9978,mseloss:0.8974
epoch [3/20], sumloss:0.9050,mseloss:0.8994
test_epoch [3/20], loss:0.8988,corr:0.9971,mseloss:0.8945
epoch [4/20], sumloss:0.9036,mseloss:0.8981
test_epoch [4/20], loss:0.8973,corr:0.9978,mseloss:0.8939
epoch [5/20], sumloss:0.9031,mseloss:0.8979
test_epoch [5/20], loss:0.8970,corr:0.9977,mseloss:0.8935
epoch [6/20], sumloss:0.9025,mseloss:0.8976
test_epoch [6/20], loss:0.8969,corr:0.9973,mseloss:0.8929
epoch [7/20], sumloss:0.9025,mseloss:0.8977
test_epoch [7/20], loss:0.8963,corr:0.9978,mseloss:0.8930
epoch [8/20], sumloss:0.9019,mseloss:0.8973
test_epoch [8/20], loss:0.8962,corr:0.9977,mseloss:0.8927
epoch [9/20], sumloss:0.9020,mseloss:0.8973
test_epoch [9/20], loss:0.8965,corr:0.9974,mseloss:0.8927
epoch [10/20], sumloss:0.9020,mseloss:0.8973
test_epoch [10/20], loss:0.8962,corr:

In [74]:
torch.save(model.state_dict(),"/mnt/cifs/result/NQ_new/bandpass/init_weight/model_initweightrun2.pth")

In [27]:
torch.save(model.state_dict(),"/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/init_weight/model_initweightrun1.pth")

In [28]:
train_dataset = MyDataset(torch.tensor(test_sample[:len(sample)],dtype=torch.float32))
train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)

test_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)

In [63]:
sample.shape

(12640, 59412)

In [29]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,5,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        spa_loss = sparse_loss(model,data)
        loss = loss + 0.0001*spa_loss
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,1.5)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:1.0299,mseloss:0.9335
test_epoch [1/20], loss:0.9247,corr:0.9994,mseloss:0.9238
epoch [2/20], sumloss:0.9276,mseloss:0.9245
test_epoch [2/20], loss:0.9235,corr:0.9993,mseloss:0.9225
epoch [3/20], sumloss:0.9253,mseloss:0.9221
test_epoch [3/20], loss:0.9214,corr:0.9991,mseloss:0.9201
epoch [4/20], sumloss:0.9228,mseloss:0.9195
test_epoch [4/20], loss:0.9187,corr:0.9992,mseloss:0.9176
epoch [5/20], sumloss:0.9211,mseloss:0.9176
test_epoch [5/20], loss:0.9162,corr:0.9993,mseloss:0.9151
epoch [6/20], sumloss:0.9190,mseloss:0.9156
test_epoch [6/20], loss:0.9129,corr:0.9992,mseloss:0.9117
epoch [7/20], sumloss:0.9167,mseloss:0.9133
test_epoch [7/20], loss:0.9110,corr:0.9991,mseloss:0.9096
epoch [8/20], sumloss:0.9160,mseloss:0.9127
test_epoch [8/20], loss:0.9105,corr:0.9992,mseloss:0.9092
epoch [9/20], sumloss:0.9158,mseloss:0.9126
test_epoch [9/20], loss:0.9106,corr:0.9990,mseloss:0.9091
epoch [10/20], sumloss:0.9156,mseloss:0.9125
test_epoch [10/20], loss:0.9102,corr:

In [31]:
torch.save(model.state_dict(),"/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/init_weight/model_initweightrun2.pth")

### 個人のデータで最適化するコード

In [45]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30
labe = ["calculation",
"default",
"emotional",
"language",
"listening","mentalizing","motor","reading","somatosensory","task","visual","working_memory"]


for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([13]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T


                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)
                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    model = MixedApproachTiedAutoEncoder(59412,node_num,bias= True)

                    mseloss = nn.MSELoss()
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)
                    param = torch.load("/mnt/cifs/result/MS_new13/model_initweight.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    model = model.to("cuda")
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(model.encoder.weight.cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    for i in range(12):
                        ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,i],label = labe[i],alpha=0.7)



                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/MS_new13/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    for i in range(12):
                        ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,i],label = labe[i],alpha=0.7)

                    
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/MS_new13/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/MS_new13/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/MS_new13/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/MS_new13/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9671,mseloss:0.9333
test_epoch [11/20], loss:0.9790,corr:0.9788,mseloss:0.9472
epoch [19/20], sumloss:0.9508,mseloss:0.9309
epoch [20/20], sumloss:0.9501,mseloss:0.9308
test_epoch [20/20], loss:0.9653,corr:0.9873,mseloss:0.9462


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9654,mseloss:0.9206
test_epoch [11/20], loss:1.0121,corr:0.9718,mseloss:0.9697
epoch [19/20], sumloss:0.9448,mseloss:0.9173
epoch [20/20], sumloss:0.9439,mseloss:0.9172
test_epoch [20/20], loss:0.9960,corr:0.9824,mseloss:0.9696


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


...

epoch [11/20], sumloss:0.8506,mseloss:0.8189
test_epoch [11/20], loss:0.9636,corr:0.9798,mseloss:0.9332
epoch [19/20], sumloss:0.8383,mseloss:0.8153
epoch [20/20], sumloss:0.8379,mseloss:0.8151
test_epoch [20/20], loss:0.9554,corr:0.9850,mseloss:0.9328


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9066,mseloss:0.8960
test_epoch [11/20], loss:0.9584,corr:0.9934,mseloss:0.9486
epoch [19/20], sumloss:0.8983,mseloss:0.8921
epoch [20/20], sumloss:0.8980,mseloss:0.8920
test_epoch [20/20], loss:0.9542,corr:0.9960,mseloss:0.9482


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9237,mseloss:0.9043
test_epoch [11/20], loss:0.9558,corr:0.9877,mseloss:0.9374
epoch [19/20], sumloss:0.9146,mseloss:0.9006
epoch [20/20], sumloss:0.9142,mseloss:0.9005
test_epoch [20/20], loss:0.9516,corr:0.9909,mseloss:0.9380





In [66]:
model

MixedApproachTiedAutoEncoder(
  (encoder): Linear(in_features=59412, out_features=5, bias=True)
  (prelu): PReLU(num_parameters=1)
)

In [51]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30

for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([6,10]):
        for is_bandpass in tqdm_notebook(["BandPassed"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T


                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)
                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    model = MixedApproachTiedAutoEncoder(59412,node_num,bias=True)

                    mseloss = nn.MSELoss()
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)
                    param = torch.load("/mnt/cifs/result/Individual_Z3/bandpass/AEnode"+str(node_num)+"/init_weight/model_initweight"+str(train_run)+".pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    model = model.to("cuda")
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19,29,39,49,59,69,79,89,99]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(model.encoder.weight.cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/Individual_Z3/bandpass/AEnode"+str(node_num)+"/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), train_encode_listlist[19].cpu().detach().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].cpu().detach().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].cpu().detach().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].cpu().detach().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/Individual_Z3/bandpass/AEnode"+str(node_num)+"/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/Individual_Z3/bandpass/AEnode"+str(node_num)+"/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/Individual_Z3/bandpass/AEnode"+str(node_num)+"/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/Individual_Z3/bandpass/AEnode"+str(node_num)+"/"+str(train_run)+
                                                                                    "/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.8573,mseloss:0.8553
test_epoch [11/20], loss:0.8696,corr:0.9987,mseloss:0.8676
epoch [19/20], sumloss:0.8496,mseloss:0.8479
epoch [20/20], sumloss:0.8493,mseloss:0.8476
test_epoch [20/20], loss:0.8707,corr:0.9989,mseloss:0.8690


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.8693,mseloss:0.8684
test_epoch [11/20], loss:0.9456,corr:0.9994,mseloss:0.9447
epoch [19/20], sumloss:0.8620,mseloss:0.8612
epoch [20/20], sumloss:0.8617,mseloss:0.8608
test_epoch [20/20], loss:0.9472,corr:0.9995,mseloss:0.9464


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


...

epoch [11/20], sumloss:0.6918,mseloss:0.6911
test_epoch [11/20], loss:0.8369,corr:0.9996,mseloss:0.8362
epoch [19/20], sumloss:0.6765,mseloss:0.6759
epoch [20/20], sumloss:0.6763,mseloss:0.6757
test_epoch [20/20], loss:0.8405,corr:0.9996,mseloss:0.8399


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.7789,mseloss:0.7781
test_epoch [11/20], loss:0.8882,corr:0.9995,mseloss:0.8875
epoch [19/20], sumloss:0.7687,mseloss:0.7680
epoch [20/20], sumloss:0.7683,mseloss:0.7676
test_epoch [20/20], loss:0.8897,corr:0.9995,mseloss:0.8890


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.7325,mseloss:0.7317
test_epoch [11/20], loss:0.8609,corr:0.9995,mseloss:0.8602
epoch [19/20], sumloss:0.7132,mseloss:0.7126
epoch [20/20], sumloss:0.7126,mseloss:0.7119
test_epoch [20/20], loss:0.8635,corr:0.9996,mseloss:0.8629





In [76]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30

for corr_alpha in tqdm_notebook([1.5,1,0.5,0.1,0.05]):
    for lasso_alpha in tqdm_notebook([1,0.001,0.005,0.01,0.05,0.1,0.5]):
        for node_num in tqdm_notebook([13]):
            for is_bandpass in tqdm_notebook(["BandPassed"]):
                for train_run in ["run1","run2"]:
                    for sub_num in use_subject_num[:use_subnum]:
                        if is_bandpass == "MSMALL":
                            test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                                "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                            test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                     "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                        else:
                            test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                            test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                     "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                        test_bandsample_100307 = test_bandsample_100307.T
                        test_bandsample_100307run2 = test_bandsample_100307run2.T


                        scaler = StandardScaler()
                        scaler.fit(test_bandsample_100307)
                        test_bandsample_100307 = scaler.transform(test_bandsample_100307)

                        scaler2 = StandardScaler()
                        scaler2.fit(test_bandsample_100307run2)
                        test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)
                        if train_run == "run1":
                            train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                            train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                            test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                            test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                        else:
                            train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                            train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                            test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                            test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                        num_epochs = 100
                        #
                        model = MixedApproachTiedAutoEncoder(59412,13,bias=True)

                        mseloss = nn.MSELoss()
                        optimizer = torch.optim.Adam(
                        model.parameters(), lr=0.0001, weight_decay=0)
                        param = torch.load("/mnt/cifs/result/NQ_new/bandpass/init_weight/model_initweight"+str(train_run)+".pth")
                        #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                        model.load_state_dict(param)
                        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                        weight_list = list()

                        encode_listlist = list()
                        train_encode_listlist = list()
                        model = model.to("cuda")
                        for epoch in range(num_epochs):
                            loss_list = list()
                            mseloss_list = list()
                            corr_list = list()
                            spa_loss_list = list()

                            test_loss_list = list()
                            test_mseloss_list = list()
                            test_corr_list = list()
                            test_spa_loss_list = list()

                            encode_list = list()
                            train_encode_list = list()
                            for data in train_dataloader:

                                data = data.to("cuda")
                                # ===================forward=====================
                                encode,output = model(data)
                                train_encode_list.append(encode)
                                loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                                spa_loss = sparse_loss(model,data)

                                loss = loss + lasso_alpha*spa_loss
                                loss_list.append(loss.item())
                                mseloss_list.append(mse_loss.item())
                                corr_list.append(corr.item())
                                spa_loss_list.append(spa_loss.item())
                                # ===================backward====================
                                optimizer.zero_grad()
                                loss.backward()
                                optimizer.step()
                            # ===================log========================
                            if epoch in [10,19,29,39,49,89,99]:
                                print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f},spaloss:{:.4f}'
                                      .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list),np.mean(spa_loss_list)))
                            train_encode_listlist.append(torch.cat(train_encode_list))









                            #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                            for data in test_dataloader:

                                data = data.to("cuda")
                                model.eval()
                                # ===================forward=====================
                                encode,output = model(data)
                                encode_list.append(encode)
                                loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                                spa_loss = sparse_loss(model,data)
                                loss = loss + lasso_alpha*spa_loss
                                test_loss_list.append(loss.item())
                                test_mseloss_list.append(mse_loss.item())
                                test_corr_list.append(corr.item())
                                test_spa_loss_list.append(spa_loss.item())
                            encode_listlist.append(torch.cat(encode_list))
                            if epoch in [10,19,29,39,49,89,99]:
                                print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f},spaloss:{:.4f}'
                                      .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list),np.mean(test_spa_loss_list)))
                            weight_list.append(model.encoder.weight.cpu().t())
                            scheduler.step(mse_loss)


                        fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,0],color='red',alpha=0.7)
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,1],color='blue',alpha=0.7)
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,2],color='black',alpha=0.7)
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,3],color='green',alpha=0.7)
                        ax.vlines([50], -160, 160, linestyles='dashed')
                        ax.vlines([150], -160, 160, linestyles='dashed')
                        ax.vlines([250], -160, 160, linestyles='dashed')
                        ax.vlines([300], -160, 160, linestyles='dashed')
                        ax.set_xlim(0, 316)
                        plt.savefig("/mnt/cifs/result/NQ_new/bandpass/sparse_alpha"+str(corr_alpha)+"_lambda"+str(lasso_alpha)+"/"+str(train_run)+"/"+
                                    str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                    "_node"+str(node_num)+"test.png")
                        plt.close()






                        fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,0],color='red',alpha=0.7)
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,1],color='blue',alpha=0.7)
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,2],color='black',alpha=0.7)
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,3],color='green',alpha=0.7)
                        ax.vlines([50], -160, 160, linestyles='dashed')
                        ax.vlines([150], -160, 160, linestyles='dashed')
                        ax.vlines([250], -160, 160, linestyles='dashed')
                        ax.vlines([300], -160, 160, linestyles='dashed')
                        ax.set_xlim(0, 316)
                        plt.savefig("/mnt/cifs/result/NQ_new/bandpass/sparse_alpha"+str(corr_alpha)+"_lambda"+str(lasso_alpha)+"/"+str(train_run)+"/"+
                                    str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                    "_node"+str(node_num)+"train.png")
                        plt.close()



                        #E = torch.load("./weight_corr0.5868.pth")
                        #F = torch.load("./encode_output.pth")
                        #G = torch.load("./weight_corr0.7828.pth")


                        pd.DataFrame(weight_list[99].cpu().detach().numpy()).to_csv("/mnt/cifs/result/NQ_new/bandpass/sparse_alpha"+str(corr_alpha)+"_lambda"+str(lasso_alpha)+"/"+str(train_run)+"/"
                                                                +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                        pd.DataFrame(train_encode_listlist[99].cpu().detach().numpy()).to_csv("/mnt/cifs/result/NQ_new/bandpass/sparse_alpha"+str(corr_alpha)+"_lambda"+str(lasso_alpha)+"/"+str(train_run)+"/"+
                                                                str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                        pd.DataFrame(encode_listlist[99].cpu().detach().numpy()).to_csv("/mnt/cifs/result/NQ_new/bandpass/sparse_alpha"+str(corr_alpha)+"_lambda"+str(lasso_alpha)+"/"+str(train_run)+"/"+
                                                                str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.6542,mseloss:1.0073,spaloss:0.4250
test_epoch [11/100], loss:6.5298,corr:0.8522,mseloss:0.9888,spaloss:5.3193
epoch [20/100], sumloss:1.3370,mseloss:1.0075,spaloss:0.1080
test_epoch [20/100], loss:6.4480,corr:0.8523,mseloss:0.9898,spaloss:5.2366
epoch [30/100], sumloss:1.2765,mseloss:1.0076,spaloss:0.0477
test_epoch [30/100], loss:6.4389,corr:0.8525,mseloss:0.9899,spaloss:5.2277
epoch [40/100], sumloss:1.2688,mseloss:1.0076,spaloss:0.0400
test_epoch [40/100], loss:6.4364,corr:0.8525,mseloss:0.9900,spaloss:5.2253
epoch [50/100], sumloss:1.2679,mseloss:1.0076,spaloss:0.0392
test_epoch [50/100], loss:6.4361,corr:0.8526,mseloss:0.9900,spaloss:5.2250
epoch [90/100], sumloss:1.2675,mseloss:1.0076,spaloss:0.0388
test_epoch [90/100], loss:6.4359,corr:0.8526,mseloss:0.9900,spaloss:5.2248
epoch [100/100], sumloss:1.2675,mseloss:1.0076,spaloss:0.0387
test_epoch [100/100], loss:6.4358,corr:0.8526,mseloss:0.9900,spaloss:5.2247


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.5922,mseloss:1.0034,spaloss:0.3896
test_epoch [11/100], loss:6.8696,corr:0.8675,mseloss:0.9967,spaloss:5.6741
epoch [20/100], sumloss:1.2915,mseloss:1.0037,spaloss:0.0913
test_epoch [20/100], loss:6.7946,corr:0.8691,mseloss:0.9973,spaloss:5.6009
epoch [30/100], sumloss:1.2313,mseloss:1.0037,spaloss:0.0317
test_epoch [30/100], loss:6.7841,corr:0.8694,mseloss:0.9974,spaloss:5.5908
epoch [40/100], sumloss:1.2240,mseloss:1.0037,spaloss:0.0245
test_epoch [40/100], loss:6.7832,corr:0.8695,mseloss:0.9974,spaloss:5.5900
epoch [50/100], sumloss:1.2234,mseloss:1.0037,spaloss:0.0240
test_epoch [50/100], loss:6.7831,corr:0.8695,mseloss:0.9974,spaloss:5.5899
epoch [90/100], sumloss:1.2232,mseloss:1.0037,spaloss:0.0239
test_epoch [90/100], loss:6.7830,corr:0.8695,mseloss:0.9974,spaloss:5.5899
epoch [100/100], sumloss:1.2232,mseloss:1.0037,spaloss:0.0238
test_epoch [100/100], loss:6.7830,corr:0.8695,mseloss:0.9974,spaloss:5.5899


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


...

epoch [11/100], sumloss:1.2600,mseloss:1.0051,spaloss:0.4930
test_epoch [11/100], loss:4.1295,corr:0.8312,mseloss:0.9856,spaloss:6.2710
epoch [20/100], sumloss:1.0717,mseloss:1.0052,spaloss:0.1161
test_epoch [20/100], loss:4.1123,corr:0.8316,mseloss:0.9859,spaloss:6.2361
epoch [30/100], sumloss:1.0277,mseloss:1.0053,spaloss:0.0281
test_epoch [30/100], loss:4.1107,corr:0.8316,mseloss:0.9859,spaloss:6.2328
epoch [40/100], sumloss:1.0197,mseloss:1.0053,spaloss:0.0121
test_epoch [40/100], loss:4.1110,corr:0.8316,mseloss:0.9859,spaloss:6.2334
epoch [50/100], sumloss:1.0191,mseloss:1.0053,spaloss:0.0108
test_epoch [50/100], loss:4.1110,corr:0.8316,mseloss:0.9859,spaloss:6.2334
epoch [90/100], sumloss:1.0190,mseloss:1.0053,spaloss:0.0106
test_epoch [90/100], loss:4.1110,corr:0.8316,mseloss:0.9859,spaloss:6.2334
epoch [100/100], sumloss:1.0190,mseloss:1.0053,spaloss:0.0106
test_epoch [100/100], loss:4.1110,corr:0.8316,mseloss:0.9859,spaloss:6.2334


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.1901,mseloss:1.0035,spaloss:0.3580
test_epoch [11/100], loss:4.2611,corr:0.8486,mseloss:0.9803,spaloss:6.5465
epoch [20/100], sumloss:1.0542,mseloss:1.0036,spaloss:0.0861
test_epoch [20/100], loss:4.2617,corr:0.8493,mseloss:0.9802,spaloss:6.5480
epoch [30/100], sumloss:1.0206,mseloss:1.0036,spaloss:0.0188
test_epoch [30/100], loss:4.2613,corr:0.8493,mseloss:0.9802,spaloss:6.5472
epoch [40/100], sumloss:1.0143,mseloss:1.0036,spaloss:0.0063
test_epoch [40/100], loss:4.2613,corr:0.8493,mseloss:0.9802,spaloss:6.5471
epoch [50/100], sumloss:1.0141,mseloss:1.0036,spaloss:0.0059
test_epoch [50/100], loss:4.2613,corr:0.8493,mseloss:0.9802,spaloss:6.5472
epoch [90/100], sumloss:1.0141,mseloss:1.0036,spaloss:0.0058
test_epoch [90/100], loss:4.2613,corr:0.8493,mseloss:0.9802,spaloss:6.5472
epoch [100/100], sumloss:1.0140,mseloss:1.0036,spaloss:0.0058
test_epoch [100/100], loss:4.2613,corr:0.8493,mseloss:0.9802,spaloss:6.5472


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.2059,mseloss:1.0023,spaloss:0.3914
test_epoch [11/100], loss:4.0214,corr:0.8411,mseloss:0.9779,spaloss:6.0711
epoch [20/100], sumloss:1.0608,mseloss:1.0024,spaloss:0.1009
test_epoch [20/100], loss:4.0114,corr:0.8411,mseloss:0.9780,spaloss:6.0509
epoch [30/100], sumloss:1.0199,mseloss:1.0024,spaloss:0.0192
test_epoch [30/100], loss:4.0117,corr:0.8412,mseloss:0.9780,spaloss:6.0516
epoch [40/100], sumloss:1.0136,mseloss:1.0024,spaloss:0.0066
test_epoch [40/100], loss:4.0116,corr:0.8412,mseloss:0.9780,spaloss:6.0513
epoch [50/100], sumloss:1.0133,mseloss:1.0024,spaloss:0.0059
test_epoch [50/100], loss:4.0116,corr:0.8412,mseloss:0.9780,spaloss:6.0513
epoch [90/100], sumloss:1.0133,mseloss:1.0024,spaloss:0.0059
test_epoch [90/100], loss:4.0116,corr:0.8412,mseloss:0.9780,spaloss:6.0514
epoch [100/100], sumloss:1.0133,mseloss:1.0024,spaloss:0.0059
test_epoch [100/100], loss:4.0116,corr:0.8412,mseloss:0.9780,spaloss:6.0514






In [34]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30

for corr_alpha in tqdm_notebook([1.5]):
    for lasso_alpha in tqdm_notebook([0.05]):
        for node_num in tqdm_notebook([5]):
            for is_bandpass in tqdm_notebook(["BandPassed"]):
                for train_run in ["run1","run2"]:
                    for sub_num in use_subject_num[:use_subnum]:
                        if is_bandpass == "MSMALL":
                            test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                                "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                            test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                     "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                        else:
                            test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                            test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                     "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                        test_bandsample_100307 = test_bandsample_100307.T
                        test_bandsample_100307run2 = test_bandsample_100307run2.T


                        scaler = StandardScaler()
                        scaler.fit(test_bandsample_100307)
                        test_bandsample_100307 = scaler.transform(test_bandsample_100307)

                        scaler2 = StandardScaler()
                        scaler2.fit(test_bandsample_100307run2)
                        test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)
                        if train_run == "run1":
                            train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                            train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                            test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                            test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                        else:
                            train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                            train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                            test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                            test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                        num_epochs = 100
                        #
                        model = MixedApproachTiedAutoEncoder(59412,5,bias=True)

                        mseloss = nn.MSELoss()
                        optimizer = torch.optim.Adam(
                        model.parameters(), lr=0.0001, weight_decay=0)
                        param = torch.load("/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/init_weight/model_initweight"+str(train_run)+".pth")
                        #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                        model.load_state_dict(param)
                        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                        weight_list = list()

                        encode_listlist = list()
                        train_encode_listlist = list()
                        model = model.to("cuda")
                        for epoch in range(num_epochs):
                            loss_list = list()
                            mseloss_list = list()
                            corr_list = list()
                            spa_loss_list = list()

                            test_loss_list = list()
                            test_mseloss_list = list()
                            test_corr_list = list()
                            test_spa_loss_list = list()

                            encode_list = list()
                            train_encode_list = list()
                            for data in train_dataloader:

                                data = data.to("cuda")
                                # ===================forward=====================
                                encode,output = model(data)
                                train_encode_list.append(encode)
                                loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                                spa_loss = sparse_loss(model,data)

                                loss = loss + lasso_alpha*spa_loss
                                loss_list.append(loss.item())
                                mseloss_list.append(mse_loss.item())
                                corr_list.append(corr.item())
                                spa_loss_list.append(spa_loss.item())
                                # ===================backward====================
                                optimizer.zero_grad()
                                loss.backward()
                                optimizer.step()
                            # ===================log========================
                            if epoch in [10,19,29,39,49,89,99]:
                                print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f},spaloss:{:.4f}'
                                      .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list),np.mean(spa_loss_list)))
                            train_encode_listlist.append(torch.cat(train_encode_list))









                            #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                            for data in test_dataloader:

                                data = data.to("cuda")
                                model.eval()
                                # ===================forward=====================
                                encode,output = model(data)
                                encode_list.append(encode)
                                loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                                spa_loss = sparse_loss(model,data)
                                loss = loss + lasso_alpha*spa_loss
                                test_loss_list.append(loss.item())
                                test_mseloss_list.append(mse_loss.item())
                                test_corr_list.append(corr.item())
                                test_spa_loss_list.append(spa_loss.item())
                            encode_listlist.append(torch.cat(encode_list))
                            if epoch in [10,19,29,39,49,89,99]:
                                print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f},spaloss:{:.4f}'
                                      .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list),np.mean(test_spa_loss_list)))
                            weight_list.append(model.encoder.weight.cpu().t())
                            scheduler.step(mse_loss)


                        fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,0],color='red',alpha=0.7)
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,1],color='blue',alpha=0.7)
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,2],color='black',alpha=0.7)
                        ax.plot(np.arange(316), encode_listlist[99].cpu().detach().numpy()[:,3],color='green',alpha=0.7)
                        ax.vlines([50], -160, 160, linestyles='dashed')
                        ax.vlines([150], -160, 160, linestyles='dashed')
                        ax.vlines([250], -160, 160, linestyles='dashed')
                        ax.vlines([300], -160, 160, linestyles='dashed')
                        ax.set_xlim(0, 316)
                        plt.savefig("/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/"+str(train_run)+"/"+
                                    str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                    "_node"+str(node_num)+"test.png")
                        plt.close()






                        fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,0],color='red',alpha=0.7)
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,1],color='blue',alpha=0.7)
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,2],color='black',alpha=0.7)
                        ax.plot(np.arange(316), train_encode_listlist[99].cpu().detach().numpy()[:,3],color='green',alpha=0.7)
                        ax.vlines([50], -160, 160, linestyles='dashed')
                        ax.vlines([150], -160, 160, linestyles='dashed')
                        ax.vlines([250], -160, 160, linestyles='dashed')
                        ax.vlines([300], -160, 160, linestyles='dashed')
                        ax.set_xlim(0, 316)
                        plt.savefig("/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/"+str(train_run)+"/"+
                                    str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                    "_node"+str(node_num)+"train.png")
                        plt.close()



                        #E = torch.load("./weight_corr0.5868.pth")
                        #F = torch.load("./encode_output.pth")
                        #G = torch.load("./weight_corr0.7828.pth")


                        pd.DataFrame(weight_list[99].cpu().detach().numpy()).to_csv("/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/"+str(train_run)+"/"
                                                                +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                        pd.DataFrame(train_encode_listlist[99].cpu().detach().numpy()).to_csv("/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/"+str(train_run)+"/"+
                                                                str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                        pd.DataFrame(encode_listlist[99].cpu().detach().numpy()).to_csv("/mnt/cifs/result/Individual_Z3/bandpass/sparse_alpha1.5_lambda0.05/"+str(train_run)+"/"+
                                                                str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.2490,mseloss:0.9994,spaloss:3.3709
test_epoch [11/100], loss:1.5058,corr:0.9452,mseloss:0.9852,spaloss:8.7688
epoch [20/100], sumloss:1.1898,mseloss:1.0043,spaloss:1.9367
test_epoch [20/100], loss:1.4469,corr:0.9408,mseloss:0.9951,spaloss:7.2608
epoch [30/100], sumloss:1.1783,mseloss:1.0051,spaloss:1.6650
test_epoch [30/100], loss:1.4339,corr:0.9400,mseloss:0.9967,spaloss:6.9459
epoch [40/100], sumloss:1.1764,mseloss:1.0052,spaloss:1.6208
test_epoch [40/100], loss:1.4316,corr:0.9399,mseloss:0.9969,spaloss:6.8900
epoch [50/100], sumloss:1.1760,mseloss:1.0052,spaloss:1.6129
test_epoch [50/100], loss:1.4311,corr:0.9399,mseloss:0.9970,spaloss:6.8800
epoch [90/100], sumloss:1.1758,mseloss:1.0053,spaloss:1.6083
test_epoch [90/100], loss:1.4309,corr:0.9399,mseloss:0.9970,spaloss:6.8735
epoch [100/100], sumloss:1.1758,mseloss:1.0053,spaloss:1.6072
test_epoch [100/100], loss:1.4308,corr:0.9399,mseloss:0.9970,spaloss:6.8720


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.2220,mseloss:0.9948,spaloss:3.3883
test_epoch [11/100], loss:1.4752,corr:0.9609,mseloss:0.9977,spaloss:8.3791
epoch [20/100], sumloss:1.1612,mseloss:1.0003,spaloss:1.9434
test_epoch [20/100], loss:1.4291,corr:0.9575,mseloss:1.0023,spaloss:7.2599
epoch [30/100], sumloss:1.1479,mseloss:1.0012,spaloss:1.6375
test_epoch [30/100], loss:1.4187,corr:0.9568,mseloss:1.0031,spaloss:7.0160
epoch [40/100], sumloss:1.1456,mseloss:1.0014,spaloss:1.5872
test_epoch [40/100], loss:1.4168,corr:0.9567,mseloss:1.0032,spaloss:6.9734
epoch [50/100], sumloss:1.1452,mseloss:1.0014,spaloss:1.5779
test_epoch [50/100], loss:1.4165,corr:0.9567,mseloss:1.0032,spaloss:6.9658
epoch [90/100], sumloss:1.1450,mseloss:1.0014,spaloss:1.5725
test_epoch [90/100], loss:1.4163,corr:0.9567,mseloss:1.0033,spaloss:6.9607
epoch [100/100], sumloss:1.1449,mseloss:1.0014,spaloss:1.5712
test_epoch [100/100], loss:1.4162,corr:0.9567,mseloss:1.0033,spaloss:6.9595


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


...

epoch [11/100], sumloss:1.1645,mseloss:1.0047,spaloss:1.3615
test_epoch [11/100], loss:1.4361,corr:0.9384,mseloss:0.9954,spaloss:6.9655
epoch [20/100], sumloss:1.1420,mseloss:1.0049,spaloss:0.7921
test_epoch [20/100], loss:1.4284,corr:0.9350,mseloss:0.9959,spaloss:6.6986
epoch [30/100], sumloss:1.1385,mseloss:1.0050,spaloss:0.7035
test_epoch [30/100], loss:1.4264,corr:0.9344,mseloss:0.9961,spaloss:6.6398
epoch [40/100], sumloss:1.1380,mseloss:1.0050,spaloss:0.6898
test_epoch [40/100], loss:1.4261,corr:0.9343,mseloss:0.9961,spaloss:6.6299
epoch [50/100], sumloss:1.1379,mseloss:1.0050,spaloss:0.6873
test_epoch [50/100], loss:1.4260,corr:0.9343,mseloss:0.9961,spaloss:6.6282
epoch [90/100], sumloss:1.1378,mseloss:1.0050,spaloss:0.6860
test_epoch [90/100], loss:1.4260,corr:0.9343,mseloss:0.9961,spaloss:6.6270
epoch [100/100], sumloss:1.1378,mseloss:1.0050,spaloss:0.6857
test_epoch [100/100], loss:1.4260,corr:0.9343,mseloss:0.9961,spaloss:6.6267


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.1388,mseloss:1.0032,spaloss:1.0259
test_epoch [11/100], loss:1.4379,corr:0.9432,mseloss:0.9918,spaloss:7.2192
epoch [20/100], sumloss:1.1218,mseloss:1.0034,spaloss:0.5684
test_epoch [20/100], loss:1.4328,corr:0.9399,mseloss:0.9922,spaloss:7.0077
epoch [30/100], sumloss:1.1191,mseloss:1.0034,spaloss:0.4961
test_epoch [30/100], loss:1.4322,corr:0.9394,mseloss:0.9923,spaloss:6.9784
epoch [40/100], sumloss:1.1188,mseloss:1.0034,spaloss:0.4863
test_epoch [40/100], loss:1.4321,corr:0.9393,mseloss:0.9923,spaloss:6.9742
epoch [50/100], sumloss:1.1187,mseloss:1.0034,spaloss:0.4850
test_epoch [50/100], loss:1.4320,corr:0.9393,mseloss:0.9923,spaloss:6.9734
epoch [90/100], sumloss:1.1187,mseloss:1.0034,spaloss:0.4845
test_epoch [90/100], loss:1.4320,corr:0.9393,mseloss:0.9923,spaloss:6.9730
epoch [100/100], sumloss:1.1187,mseloss:1.0034,spaloss:0.4844
test_epoch [100/100], loss:1.4320,corr:0.9393,mseloss:0.9923,spaloss:6.9729


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/100], sumloss:1.1581,mseloss:1.0017,spaloss:1.2052
test_epoch [11/100], loss:1.4212,corr:0.9354,mseloss:0.9894,spaloss:6.6999
epoch [20/100], sumloss:1.1385,mseloss:1.0021,spaloss:0.6776
test_epoch [20/100], loss:1.4124,corr:0.9316,mseloss:0.9900,spaloss:6.3948
epoch [30/100], sumloss:1.1356,mseloss:1.0021,spaloss:0.6009
test_epoch [30/100], loss:1.4116,corr:0.9310,mseloss:0.9900,spaloss:6.3615
epoch [40/100], sumloss:1.1351,mseloss:1.0021,spaloss:0.5880
test_epoch [40/100], loss:1.4115,corr:0.9309,mseloss:0.9901,spaloss:6.3555
epoch [50/100], sumloss:1.1351,mseloss:1.0021,spaloss:0.5864
test_epoch [50/100], loss:1.4115,corr:0.9309,mseloss:0.9901,spaloss:6.3545
epoch [90/100], sumloss:1.1351,mseloss:1.0021,spaloss:0.5856
test_epoch [90/100], loss:1.4114,corr:0.9309,mseloss:0.9901,spaloss:6.3537
epoch [100/100], sumloss:1.1351,mseloss:1.0021,spaloss:0.5854
test_epoch [100/100], loss:1.4114,corr:0.9309,mseloss:0.9901,spaloss:6.3536






In [71]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,13,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:0.8857,mseloss:0.8800
test_epoch [1/20], loss:0.8684,corr:0.5075,mseloss:0.8635
epoch [2/20], sumloss:0.8491,mseloss:0.8439
test_epoch [2/20], loss:0.8623,corr:0.4681,mseloss:0.8570
epoch [3/20], sumloss:0.8463,mseloss:0.8409
test_epoch [3/20], loss:0.8619,corr:0.4689,mseloss:0.8566
epoch [4/20], sumloss:0.8459,mseloss:0.8406
test_epoch [4/20], loss:0.8616,corr:0.4729,mseloss:0.8563
epoch [5/20], sumloss:0.8457,mseloss:0.8405
test_epoch [5/20], loss:0.8615,corr:0.4809,mseloss:0.8563
epoch [6/20], sumloss:0.8456,mseloss:0.8404
test_epoch [6/20], loss:0.8616,corr:0.4852,mseloss:0.8564
epoch [7/20], sumloss:0.8455,mseloss:0.8404
test_epoch [7/20], loss:0.8612,corr:0.4913,mseloss:0.8561
epoch [8/20], sumloss:0.8454,mseloss:0.8404
test_epoch [8/20], loss:0.8615,corr:0.4940,mseloss:0.8564
epoch [9/20], sumloss:0.8453,mseloss:0.8403
test_epoch [9/20], loss:0.8615,corr:0.4968,mseloss:0.8565
epoch [10/20], sumloss:0.8453,mseloss:0.8403
test_epoch [10/20], loss:0.8613,corr:

In [72]:
torch.save(model.state_dict(),"/mnt/cifs/result/band_new13/model_initweight.pth")

In [73]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30
labe = ["calculation",
"default",
"emotional",
"language",
"listening","mentalizing","motor","reading","somatosensory","task","visual","working_memory"]


for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([13]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T


                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)
                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    model = MixedApproachTiedAutoEncoder(59412,node_num,bias= True)

                    mseloss = nn.MSELoss()
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)
                    param = torch.load("/mnt/cifs/result/band_new13/model_initweight.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    model = model.to("cuda")
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(model.encoder.weight.cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    for i in range(12):
                        ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,i],label = labe[i],alpha=0.7)



                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/band_new13/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    for i in range(12):
                        ax.plot(np.arange(316), encode_listlist[19].cpu().detach().numpy()[:,i],label = labe[i],alpha=0.7)

                    
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/band_new13/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/band_new13/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/band_new13/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/band_new13/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9685,mseloss:0.9332
test_epoch [11/20], loss:0.9888,corr:0.9779,mseloss:0.9556
epoch [19/20], sumloss:0.9522,mseloss:0.9314
epoch [20/20], sumloss:0.9516,mseloss:0.9313
test_epoch [20/20], loss:0.9759,corr:0.9867,mseloss:0.9559


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9651,mseloss:0.9197
test_epoch [11/20], loss:1.0121,corr:0.9712,mseloss:0.9689
epoch [19/20], sumloss:0.9456,mseloss:0.9165
epoch [20/20], sumloss:0.9449,mseloss:0.9164
test_epoch [20/20], loss:0.9970,corr:0.9812,mseloss:0.9688


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


...

epoch [11/20], sumloss:0.8615,mseloss:0.8185
test_epoch [11/20], loss:0.9758,corr:0.9723,mseloss:0.9342
epoch [19/20], sumloss:0.8496,mseloss:0.8163
epoch [20/20], sumloss:0.8491,mseloss:0.8162
test_epoch [20/20], loss:0.9671,corr:0.9782,mseloss:0.9345


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9166,mseloss:0.8981
test_epoch [11/20], loss:0.9675,corr:0.9884,mseloss:0.9501
epoch [19/20], sumloss:0.9056,mseloss:0.8942
epoch [20/20], sumloss:0.9052,mseloss:0.8940
test_epoch [20/20], loss:0.9611,corr:0.9927,mseloss:0.9501


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [11/20], sumloss:0.9346,mseloss:0.9113
test_epoch [11/20], loss:0.9662,corr:0.9851,mseloss:0.9438
epoch [19/20], sumloss:0.9281,mseloss:0.9094
epoch [20/20], sumloss:0.9279,mseloss:0.9093
test_epoch [20/20], loss:0.9625,corr:0.9877,mseloss:0.9440





In [170]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30



for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([5]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T

                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)

                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    model = MixedApproachTiedAutoEncoder(59412,node_num,bias= True)
                    #model = autoencoder()
                    mseloss = nn.MSELoss()
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)

                    #weightdict = model.state_dict()
                    #weightdict['encoder.weight']= torch.load("./bias/weight_corr0.4021.pth").t()
                    #初期値を作る
                    
                    #初期値weightをifで分岐
                    param = torch.load("/mnt/cifs/koyama_space/result/Individual_Z2/init_weight/model_initweight_node5.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(model.encoder.weight.cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/koyama_space/result/Individual_Z2/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/koyama_space/result/Individual_Z2/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].detach().numpy()).to_csv("/mnt/cifs/koyama_space/result/Individual_Z2/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].detach().numpy()).to_csv("/mnt/cifs/koyama_space/result/Individual_Z2/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].detach().numpy()).to_csv("/mnt/cifs/koyama_space/result/Individual_Z2/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

epoch [11/20], sumloss:0.9689,mseloss:0.9448
test_epoch [11/20], loss:0.9811,corr:0.9851,mseloss:0.9588
epoch [19/20], sumloss:0.9561,mseloss:0.9439
epoch [20/20], sumloss:0.9556,mseloss:0.9439
test_epoch [20/20], loss:0.9715,corr:0.9923,mseloss:0.9600
epoch [11/20], sumloss:0.9866,mseloss:0.9619
test_epoch [11/20], loss:0.9976,corr:0.9847,mseloss:0.9747
epoch [19/20], sumloss:0.9713,mseloss:0.9587
epoch [20/20], sumloss:0.9707,mseloss:0.9586
test_epoch [20/20], loss:0.9882,corr:0.9921,mseloss:0.9763
epoch [11/20], sumloss:0.9827,mseloss:0.9587
test_epoch [11/20], loss:0.9845,corr:0.9851,mseloss:0.9622
epoch [19/20], sumloss:0.9705,mseloss:0.9582
epoch [20/20], sumloss:0.9701,mseloss:0.9582
test_epoch [20/20], loss:0.9754,corr:0.9922,mseloss:0.9638
epoch [11/20], sumloss:0.9154,mseloss:0.8889
test_epoch [11/20], loss:0.9966,corr:0.9835,mseloss:0.9719
epoch [19/20], sumloss:0.8932,mseloss:0.8788
epoch [20/20], sumloss:0.8923,mseloss:0.8784
test_epoch [20/20], loss:0.9889,corr:0.9909,mse

epoch [19/20], sumloss:0.8938,mseloss:0.8916
epoch [20/20], sumloss:0.8936,mseloss:0.8915
test_epoch [20/20], loss:0.9835,corr:0.9986,mseloss:0.9814
epoch [11/20], sumloss:0.9364,mseloss:0.9310
test_epoch [11/20], loss:0.9669,corr:0.9967,mseloss:0.9619
epoch [19/20], sumloss:0.9321,mseloss:0.9295
epoch [20/20], sumloss:0.9320,mseloss:0.9294
test_epoch [20/20], loss:0.9656,corr:0.9983,mseloss:0.9631
epoch [11/20], sumloss:0.9139,mseloss:0.9083
test_epoch [11/20], loss:0.9176,corr:0.9965,mseloss:0.9123
epoch [19/20], sumloss:0.9115,mseloss:0.9082
epoch [20/20], sumloss:0.9114,mseloss:0.9082
test_epoch [20/20], loss:0.9160,corr:0.9979,mseloss:0.9129
epoch [11/20], sumloss:0.6861,mseloss:0.6468
test_epoch [11/20], loss:0.9958,corr:0.9741,mseloss:0.9570
epoch [19/20], sumloss:0.6782,mseloss:0.6427
epoch [20/20], sumloss:0.6780,mseloss:0.6425
test_epoch [20/20], loss:0.9932,corr:0.9764,mseloss:0.9577
epoch [11/20], sumloss:0.9387,mseloss:0.9344
test_epoch [11/20], loss:0.9823,corr:0.9974,mse

In [160]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,5,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.001, weight_decay=0)


weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data.to("cuda")
        
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

epoch [1/20], sumloss:0.9482,mseloss:0.9432
test_epoch [1/20], loss:0.9455,corr:0.4914,mseloss:0.9404
epoch [2/20], sumloss:0.9462,mseloss:0.9412
test_epoch [2/20], loss:0.9462,corr:0.4813,mseloss:0.9410
epoch [3/20], sumloss:0.9463,mseloss:0.9412
test_epoch [3/20], loss:0.9459,corr:0.4818,mseloss:0.9407
epoch [4/20], sumloss:0.9462,mseloss:0.9411
test_epoch [4/20], loss:0.9453,corr:0.5144,mseloss:0.9404
epoch [5/20], sumloss:0.9462,mseloss:0.9411
test_epoch [5/20], loss:0.9450,corr:0.5166,mseloss:0.9402
epoch [6/20], sumloss:0.9462,mseloss:0.9412
test_epoch [6/20], loss:0.9459,corr:0.5037,mseloss:0.9410
epoch [7/20], sumloss:0.9462,mseloss:0.9412
test_epoch [7/20], loss:0.9463,corr:0.4768,mseloss:0.9411
epoch [8/20], sumloss:0.9463,mseloss:0.9412
test_epoch [8/20], loss:0.9456,corr:0.4944,mseloss:0.9405
epoch [9/20], sumloss:0.9464,mseloss:0.9414
test_epoch [9/20], loss:0.9454,corr:0.4794,mseloss:0.9402
epoch [10/20], sumloss:0.9462,mseloss:0.9411
test_epoch [10/20], loss:0.9451,corr:

In [161]:
torch.save(model.state_dict(),"/mnt/cifs/koyama_space/result/Individual_Z3/init_weight/trainis_run1model_initweight_node5.pth")

In [165]:
test_sample[:12640].shape

(12640, 59412)

In [164]:
train_dataset = MyDataset(torch.tensor(test_sample[:12640],dtype=torch.float32))
train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)

test_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=True)

In [166]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,5,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.001, weight_decay=0)


weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

epoch [1/20], sumloss:0.9475,mseloss:0.9422
test_epoch [1/20], loss:0.9500,corr:0.4276,mseloss:0.9443
epoch [2/20], sumloss:0.9452,mseloss:0.9394
test_epoch [2/20], loss:0.9526,corr:0.4509,mseloss:0.9471
epoch [3/20], sumloss:0.9453,mseloss:0.9400
test_epoch [3/20], loss:0.9503,corr:0.4794,mseloss:0.9451
epoch [4/20], sumloss:0.9447,mseloss:0.9393
test_epoch [4/20], loss:0.9492,corr:0.4668,mseloss:0.9438
epoch [5/20], sumloss:0.9447,mseloss:0.9393
test_epoch [5/20], loss:0.9502,corr:0.4489,mseloss:0.9447
epoch [6/20], sumloss:0.9447,mseloss:0.9393
test_epoch [6/20], loss:0.9501,corr:0.4467,mseloss:0.9445
epoch [7/20], sumloss:0.9445,mseloss:0.9391
test_epoch [7/20], loss:0.9506,corr:0.4365,mseloss:0.9449
epoch [8/20], sumloss:0.9449,mseloss:0.9394
test_epoch [8/20], loss:0.9508,corr:0.4412,mseloss:0.9453
epoch [9/20], sumloss:0.9446,mseloss:0.9392
test_epoch [9/20], loss:0.9505,corr:0.4521,mseloss:0.9450
epoch [10/20], sumloss:0.9446,mseloss:0.9392
test_epoch [10/20], loss:0.9503,corr:

In [167]:
torch.save(model.state_dict(),"/mnt/cifs/koyama_space/result/Individual_Z3/init_weight/trainis_run2model_initweight_node5.pth")

In [169]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30



for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([5]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T

                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)

                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    model = MixedApproachTiedAutoEncoder(59412,node_num,bias= True)
                    #model = autoencoder()
                    mseloss = nn.MSELoss()
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)

                    #weightdict = model.state_dict()
                    #weightdict['encoder.weight']= torch.load("./bias/weight_corr0.4021.pth").t()
                    #初期値を作る
                    
                    #初期値weightをifで分岐
                    param = torch.load("/mnt/cifs/koyama_space/result/Individual_Z3/init_weight/trainis_"+str(train_run)+"model_initweight_node5.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight.cpu().t(),metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(model.encoder.weight.cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/koyama_space/result/Individual_Z3/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/koyama_space/result/Individual_Z3/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].detach().numpy()).to_csv("/mnt/cifs/koyama_space/result/Individual_Z3/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].detach().numpy()).to_csv("/mnt/cifs/koyama_space/result/Individual_Z3/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].detach().numpy()).to_csv("/mnt/cifs/koyama_space/result/Individual_Z3/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

epoch [11/20], sumloss:0.9730,mseloss:0.9449
test_epoch [11/20], loss:0.9850,corr:0.9827,mseloss:0.9590
epoch [19/20], sumloss:0.9583,mseloss:0.9440
epoch [20/20], sumloss:0.9577,mseloss:0.9440
test_epoch [20/20], loss:0.9737,corr:0.9910,mseloss:0.9602
epoch [11/20], sumloss:0.9886,mseloss:0.9597
test_epoch [11/20], loss:1.0027,corr:0.9821,mseloss:0.9759
epoch [19/20], sumloss:0.9706,mseloss:0.9559
epoch [20/20], sumloss:0.9699,mseloss:0.9557
test_epoch [20/20], loss:0.9918,corr:0.9907,mseloss:0.9778
epoch [11/20], sumloss:0.9871,mseloss:0.9592
test_epoch [11/20], loss:0.9882,corr:0.9827,mseloss:0.9623
epoch [19/20], sumloss:0.9731,mseloss:0.9588
epoch [20/20], sumloss:0.9726,mseloss:0.9588
test_epoch [20/20], loss:0.9773,corr:0.9910,mseloss:0.9637
epoch [11/20], sumloss:0.9202,mseloss:0.8897
test_epoch [11/20], loss:1.0005,corr:0.9811,mseloss:0.9721
epoch [19/20], sumloss:0.8961,mseloss:0.8797
epoch [20/20], sumloss:0.8952,mseloss:0.8793
test_epoch [20/20], loss:0.9911,corr:0.9896,mse

epoch [19/20], sumloss:0.8844,mseloss:0.8819
epoch [20/20], sumloss:0.8842,mseloss:0.8818
test_epoch [20/20], loss:0.9761,corr:0.9984,mseloss:0.9737
epoch [11/20], sumloss:0.9514,mseloss:0.9436
test_epoch [11/20], loss:0.9625,corr:0.9950,mseloss:0.9550
epoch [19/20], sumloss:0.9454,mseloss:0.9402
epoch [20/20], sumloss:0.9452,mseloss:0.9400
test_epoch [20/20], loss:0.9606,corr:0.9966,mseloss:0.9555
epoch [11/20], sumloss:0.9127,mseloss:0.9066
test_epoch [11/20], loss:0.9169,corr:0.9962,mseloss:0.9113
epoch [19/20], sumloss:0.9078,mseloss:0.9043
epoch [20/20], sumloss:0.9077,mseloss:0.9042
test_epoch [20/20], loss:0.9138,corr:0.9977,mseloss:0.9103
epoch [11/20], sumloss:0.6605,mseloss:0.6391
test_epoch [11/20], loss:0.9409,corr:0.9862,mseloss:0.9201
epoch [19/20], sumloss:0.6517,mseloss:0.6339
epoch [20/20], sumloss:0.6514,mseloss:0.6337
test_epoch [20/20], loss:0.9377,corr:0.9883,mseloss:0.9201
epoch [11/20], sumloss:0.9229,mseloss:0.9184
test_epoch [11/20], loss:0.9799,corr:0.9972,mse

# stacked autoencoder

In [24]:
group1_sub_path_run1 = group1_sub_path[group1_sub_path.str.contains("run1")] #run1のファイル名パス
group1_sub_path_run2 = group1_sub_path[group1_sub_path.str.contains("run2")]

sample = pd.read_csv(DATA_PATH+group1_sub_path_run1.iloc[0],sep='\t',header=None)
sample = sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(sample)
sample = scaler.transform(sample)
sample = sample[None,:,:]
for subnum in tqdm_notebook(np.sort(group1_sub_path_run1.values[1:40])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    sample = np.append(sample,A,axis=0)


test_sample = pd.read_csv(DATA_PATH+group1_sub_path_run2.iloc[0],sep='\t',header=None)
test_sample = test_sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(test_sample)
test_sample = scaler.transform(test_sample)
test_sample = test_sample[None,:,:]

for subnum in tqdm_notebook(np.sort(group1_sub_path_run2.values[1:80])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    test_sample = np.append(test_sample,A,axis=0)
sample = np.reshape(sample,(sample.shape[0]*316,-1))
test_sample = np.reshape(test_sample,(test_sample.shape[0]*316,-1))

In [29]:
sample = np.append(sample,test_sample,axis=0)

In [30]:
sample.shape

(37920, 59412)

In [28]:
test_sample.shape

(25280, 59412)

In [31]:

train_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)


test_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)

In [37]:
class MixedApproachTiedAutoEncoder(nn.Module):
    def __init__(self, inp, out,bias=False):
        super().__init__()
        seed_everything(SEED)
        self.encoder = nn.Linear(inp, out, bias=bias)

    def forward(self, input):
        encoded_feats = self.encoder(input)
        reconstructed_output = F.linear(encoded_feats, self.encoder.weight.t())
        return encoded_feats, reconstructed_output

In [87]:
def swish(x):
    return x * F.sigmoid(x)

In [42]:
class MixedApproachTiedMultiAutoEncoder(nn.Module):
    def __init__(self,xx,out,bias=False):
        super().__init__()
        seed_everything(SEED)
        self.encoder512 = nn.Linear(59412, 512, bias=True)
        self.encoder5 = nn.Linear(xx, out, bias=bias)
        self.bias512 = nn.Parameter(torch.rand(512)*0.001)
        self.bias59412 = nn.Parameter(torch.rand(59412)*0.001)
        self.relu = nn.PReLU()
        self.relu2 = nn.PReLU()
    def forward(self, input):
        encoded_feats = self.relu(self.encoder512(input))
        encoded_feats = self.encoder5(encoded_feats)
        reconstructed_output = self.relu2(F.linear(encoded_feats, self.encoder5.weight.t()) + self.bias512)
        reconstructed_output = F.linear(reconstructed_output, self.encoder512.weight.t()) + self.bias59412
        
        return encoded_feats, reconstructed_output

In [64]:

num_epochs = 20
#

#model = autoencoder()
mseloss = nn.MSELoss()

model = MixedApproachTiedMultiAutoEncoder(59412,512,5,bias=True)
model = model.to("cuda")
optimizer = torch.optim.Adam(
model.parameters(), lr=0.00001, weight_decay=0)


weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in test_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder2.weight,model.encoder.weight)
                                      ,metanalysis,0.0)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in train_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder2.weight,model.encoder.weight)
                                     ,metanalysis,0.0)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(torch.mm(model.encoder2.weight,model.encoder.weight).t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:0.9527,mseloss:0.9527
test_epoch [1/20], loss:0.9408,corr:0.0028,mseloss:0.9408
epoch [2/20], sumloss:0.9352,mseloss:0.9352
test_epoch [2/20], loss:0.9337,corr:0.0380,mseloss:0.9337
epoch [3/20], sumloss:0.9298,mseloss:0.9298
test_epoch [3/20], loss:0.9322,corr:0.0592,mseloss:0.9322
epoch [4/20], sumloss:0.9287,mseloss:0.9287
test_epoch [4/20], loss:0.9320,corr:0.0555,mseloss:0.9320
epoch [5/20], sumloss:0.9282,mseloss:0.9282
test_epoch [5/20], loss:0.9318,corr:0.0533,mseloss:0.9318
epoch [6/20], sumloss:0.9279,mseloss:0.9279
test_epoch [6/20], loss:0.9317,corr:0.0521,mseloss:0.9317
epoch [7/20], sumloss:0.9276,mseloss:0.9276
test_epoch [7/20], loss:0.9317,corr:0.0515,mseloss:0.9317
epoch [8/20], sumloss:0.9274,mseloss:0.9274
test_epoch [8/20], loss:0.9315,corr:0.0513,mseloss:0.9315
epoch [9/20], sumloss:0.9271,mseloss:0.9271
test_epoch [9/20], loss:0.9316,corr:0.0514,mseloss:0.9316
epoch [10/20], sumloss:0.9270,mseloss:0.9270
test_epoch [10/20], loss:0.9316,corr:

In [56]:
torch.mm(model.encoder2.weight,model.encoder.weight).shape

torch.Size([5, 59412])

In [49]:
class MixedApproachTiedMultiAutoEncoder(nn.Module):
    def __init__(self,xx,out,model,bias=False):
        super().__init__()
        seed_everything(SEED)
        self.encoder512 = model.encoder
        self.encoder5 = nn.Linear(xx, out, bias=bias)
        self.relu = nn.PReLU()
        self.relu2 = nn.PReLU()
    def forward(self, input):
        encoded_feats = self.relu(self.encoder512(input))
        encoded_feats = self.encoder5(encoded_feats)
        reconstructed_output = self.relu2(F.linear(encoded_feats, self.encoder5.weight.t()))
        reconstructed_output = F.linear(reconstructed_output, self.encoder512.weight.t())
        
        return encoded_feats, reconstructed_output

In [48]:
MixedApproachTiedMultiAutoEncoder(512,5,bias=True)

MixedApproachTiedMultiAutoEncoder(
  (encoder512): Linear(in_features=59412, out_features=512, bias=True)
  (encoder5): Linear(in_features=512, out_features=5, bias=True)
  (relu): PReLU(num_parameters=1)
  (relu2): PReLU(num_parameters=1)
)

In [22]:
def loss_fuc(x,output,weight,metanalysis,alpha):
    """weight 59412x4"""
    mseloss = nn.MSELoss()
    
    num = len(metanalysis)
    weightdata = weight[:num,:]
    meta_std = torch.std(metanalysis,dim=1)
    weight_std = torch.std(weightdata,dim=1)
    meta_mean = torch.mean(metanalysis,dim=1)[:,None]
    weight_mean = torch.mean(weightdata,dim=1)[:,None]
    covar = torch.mean((weightdata - weight_mean)*torch.tensor((metanalysis-meta_mean)),dim=1)
    corr = covar/(weight_std*meta_std)
    
    corr_mean = torch.mean(corr)
    rloss = 1-corr_mean
    
    
    mse = mseloss(x,output)
    
    return mse + alpha*rloss,corr_mean ,mse

In [35]:
data

tensor([[-0.3953, -0.9829, -0.8094,  ..., -0.4596, -0.3459, -0.2286],
        [-0.0272,  0.2884, -0.9198,  ..., -0.5971, -0.3138, -0.0191],
        [-0.3966, -0.8055, -0.2168,  ..., -0.2651, -1.0180, -1.3196],
        ...,
        [ 0.1192,  0.2546,  0.5864,  ..., -0.3598,  0.9325, -0.2035],
        [-0.0920,  1.1277, -1.0083,  ...,  0.2395,  0.5995,  0.3700],
        [-0.1336, -0.3584,  0.4684,  ...,  1.5407,  0.4219,  0.6628]],
       device='cuda:0')

In [38]:

num_epochs = 20
#
model = MixedApproachTiedAutoEncoder(59412,512,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    
    output_result = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,0)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,0)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
        output_result.append(output)
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())
    

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


epoch [1/20], sumloss:0.8144,mseloss:0.8144
test_epoch [1/20], loss:1.2262,corr:-0.0370,mseloss:1.2262
epoch [2/20], sumloss:0.7440,mseloss:0.7440
test_epoch [2/20], loss:1.2453,corr:-0.0321,mseloss:1.2453


KeyboardInterrupt: 

In [66]:
torch.save(model.state_dict(),"/mnt/cifs/result/stacked/init_weight/model_initweight512pth")

In [73]:
model512 = MixedApproachTiedAutoEncoder(59412,512,bias=True)
model = MixedApproachTiedMultiAutoEncoder(512,5,model512,bias=True).cuda()


In [74]:
torch.mm(model.encoder5.weight,model.encoder512.weight).shape

torch.Size([5, 59412])

In [108]:

num_epochs = 20
#
model512 = MixedApproachTiedAutoEncoder(59412,512,bias=True)
param = torch.load("/mnt/cifs/result/stacked/init_weight/model_initweight512pth")
model512.load_state_dict(param)
model512 = model512.to("cuda")
#model = autoencoder()
mseloss = nn.MSELoss()

model = MixedApproachTiedMultiAutoEncoder(512,5,model512,bias=True)
model = model.to("cuda")
optimizer = torch.optim.Adam(
model.parameters(), lr=0.001, weight_decay=0)

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                      ,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                     ,metanalysis,1)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(torch.mm(model.encoder5.weight,model.encoder512.weight).t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [1/20], sumloss:0.9391,mseloss:0.9342
test_epoch [1/20], loss:1.3871,corr:0.5494,mseloss:0.9365
epoch [2/20], sumloss:0.9430,mseloss:0.9377
test_epoch [2/20], loss:1.3430,corr:0.5851,mseloss:0.9281
epoch [3/20], sumloss:0.9413,mseloss:0.9366
test_epoch [3/20], loss:1.4161,corr:0.5138,mseloss:0.9299
epoch [4/20], sumloss:0.9417,mseloss:0.9370
test_epoch [4/20], loss:1.5438,corr:0.3951,mseloss:0.9390
epoch [5/20], sumloss:0.9420,mseloss:0.9373
test_epoch [5/20], loss:1.3851,corr:0.5479,mseloss:0.9330
epoch [6/20], sumloss:0.9433,mseloss:0.9386
test_epoch [6/20], loss:1.3648,corr:0.5652,mseloss:0.9300
epoch [7/20], sumloss:0.9425,mseloss:0.9380
test_epoch [7/20], loss:1.4080,corr:0.5463,mseloss:0.9543
epoch [8/20], sumloss:0.9406,mseloss:0.9357
test_epoch [8/20], loss:1.3894,corr:0.5425,mseloss:0.9319
epoch [9/20], sumloss:0.9601,mseloss:0.9552
test_epoch [9/20], loss:1.3925,corr:0.5414,mseloss:0.9339
epoch [10/20], sumloss:0.9399,mseloss:0.9353
test_epoch [10/20], loss:1.3387,corr:

In [110]:
torch.save(model.state_dict(),"/mnt/cifs/result/stacked/init_weight/stackedmodel_initweight.pth")

In [91]:
output


tensor([[ 4.6944e-05, -2.1187e-04,  1.5313e-04,  ..., -1.1449e-04,
         -1.6504e-04,  8.3778e-05],
        [-7.3465e-05, -6.2338e-04,  7.7933e-05,  ..., -3.0466e-04,
         -1.4378e-04,  3.1503e-04],
        [ 1.0912e-04, -2.6869e-04, -1.1204e-04,  ..., -1.7813e-04,
          3.0927e-05,  8.2726e-05],
        ...,
        [ 6.3180e-05, -3.7709e-04,  4.0043e-04,  ..., -6.8776e-05,
         -2.9950e-04,  1.2863e-04],
        [-1.0886e-04, -7.0158e-04,  2.4234e-04,  ..., -3.9374e-04,
         -1.9463e-04,  4.3609e-04],
        [-1.0938e-04, -8.7600e-04,  3.1547e-04,  ..., -3.2176e-04,
         -2.7709e-04,  4.2992e-04]], device='cuda:0', grad_fn=<MmBackward>)

In [29]:
mse_summary = mse_summary.set_index([0])

In [39]:
mse_summary = mse_summary.rename(columns = {0:"subjectnum"})
mse_summary = mse_summary.set_index(["subjectnum"])
mse_summary["stacked"] = 0

In [43]:
mse_summary.loc[486759,"stacked"]

0

In [24]:
mse_summary = pd.read_csv("/mnt/cifs/result/mse_summary.csv")

In [26]:
mse_summary = mse_summary.set_index(["subjectnum"])

In [57]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30



for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([5]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T

                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)

                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    model512 = MixedApproachTiedAutoEncoder(59412,512,bias=False)
                    model = MixedApproachTiedMultiAutoEncoder(512,5,model512,bias=True)
                    model = model.to("cuda")
                    
                    mseloss = nn.MSELoss()

                    
                    #初期値weightをifで分岐
                    param = torch.load("/mnt/cifs/result/stacked/init_weight/stackedmodel_initweight.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                                          ,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                                         ,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(torch.mm(model.encoder5.weight,model.encoder512.weight).cpu().t())
                        scheduler.step(mse_loss)
                    mse_summary.loc[sub_num,"stacked"] = np.mean(test_mseloss_list)
"""
                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/stacked/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/stacked/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].detach().cpu().numpy()).to_csv("/mnt/cifs/result/stacked/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/stacked/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/stacked/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                    """

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9557,mseloss:0.9494
test_epoch [11/20], loss:0.9590,corr:0.9961,mseloss:0.9532
epoch [19/20], sumloss:0.9490,mseloss:0.9460
epoch [20/20], sumloss:0.9487,mseloss:0.9458
test_epoch [20/20], loss:0.9548,corr:0.9981,mseloss:0.9519


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9773,mseloss:0.9706
test_epoch [11/20], loss:0.9807,corr:0.9959,mseloss:0.9745
epoch [19/20], sumloss:0.9709,mseloss:0.9675
epoch [20/20], sumloss:0.9706,mseloss:0.9673
test_epoch [20/20], loss:0.9774,corr:0.9979,mseloss:0.9742


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


...

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9365,mseloss:0.9347
test_epoch [11/20], loss:0.9589,corr:0.9988,mseloss:0.9571
epoch [19/20], sumloss:0.9193,mseloss:0.9181
epoch [20/20], sumloss:0.9186,mseloss:0.9174
test_epoch [20/20], loss:0.9602,corr:0.9992,mseloss:0.9590


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9329,mseloss:0.9312
test_epoch [11/20], loss:0.9466,corr:0.9990,mseloss:0.9450
epoch [19/20], sumloss:0.9260,mseloss:0.9249
epoch [20/20], sumloss:0.9257,mseloss:0.9246
test_epoch [20/20], loss:0.9464,corr:0.9993,mseloss:0.9453





'\n                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,0],color=\'red\',alpha=0.7)\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,1],color=\'blue\',alpha=0.7)\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,2],color=\'black\',alpha=0.7)\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,3],color=\'green\',alpha=0.7)\n                    ax.vlines([50], -160, 160, linestyles=\'dashed\')\n                    ax.vlines([150], -160, 160, linestyles=\'dashed\')\n                    ax.vlines([250], -160, 160, linestyles=\'dashed\')\n                    ax.vlines([300], -160, 160, linestyles=\'dashed\')\n                    ax.set_xlim(0, 316)\n                    plt.savefig("/mnt/cifs/result/stacked/"+str(train_run)+"/"+\n                                st

In [59]:
mse_summary.to_csv("/mnt/cifs/result/mse_summary.csv")

In [51]:
param

OrderedDict([('encoder512.weight',
              tensor([[ 0.0398, -0.0263,  0.0213,  ...,  0.0801,  0.1057,  0.0859],
                      [-0.0102,  0.0226, -0.0050,  ...,  0.0433, -0.0149,  0.0483],
                      [-0.0674, -0.0065,  0.0101,  ..., -0.0587, -0.0494, -0.0164],
                      ...,
                      [-0.0076,  0.0463, -0.0196,  ..., -0.0040, -0.0684, -0.0397],
                      [ 0.0128, -0.0087, -0.0286,  ...,  0.0300,  0.0588,  0.0466],
                      [-0.0330, -0.0016, -0.0899,  ...,  0.0196,  0.1038, -0.0005]],
                     device='cuda:0')),
             ('encoder5.weight',
              tensor([[ 4.3227e-03, -1.2106e-03,  5.5805e-03,  ...,  1.7494e-03,
                       -2.0870e-03, -1.7578e-03],
                      [-1.1718e-03, -1.8988e-03, -2.9000e-05,  ..., -1.2796e-03,
                       -1.3590e-03, -8.8340e-04],
                      [-2.2759e-03,  4.0177e-04,  5.2467e-04,  ..., -5.7019e-04,
                 

In [91]:

num_epochs = 20
#

#model = autoencoder()
mseloss = nn.MSELoss()

model = MixedApproachTiedMultiAutoEncoder(512,5,bias=True)
model = model.to("cuda")
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                      ,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                     ,metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(torch.mm(model.encoder5.weight,model.encoder512.weight).t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [1/20], sumloss:0.9373,mseloss:0.9330
test_epoch [1/20], loss:0.9442,corr:0.5377,mseloss:0.9395
epoch [2/20], sumloss:0.9292,mseloss:0.9247
test_epoch [2/20], loss:0.9430,corr:0.5538,mseloss:0.9386
epoch [3/20], sumloss:0.9274,mseloss:0.9230
test_epoch [3/20], loss:0.9434,corr:0.5553,mseloss:0.9390
epoch [4/20], sumloss:0.9263,mseloss:0.9220
test_epoch [4/20], loss:0.9430,corr:0.5673,mseloss:0.9387
epoch [5/20], sumloss:0.9255,mseloss:0.9212
test_epoch [5/20], loss:0.9432,corr:0.5725,mseloss:0.9389
epoch [6/20], sumloss:0.9245,mseloss:0.9202
test_epoch [6/20], loss:0.9432,corr:0.5856,mseloss:0.9390
epoch [7/20], sumloss:0.9238,mseloss:0.9196
test_epoch [7/20], loss:0.9438,corr:0.5784,mseloss:0.9396
epoch [8/20], sumloss:0.9232,mseloss:0.9190
test_epoch [8/20], loss:0.9437,corr:0.5962,mseloss:0.9396
epoch [9/20], sumloss:0.9223,mseloss:0.9183
test_epoch [9/20], loss:0.9438,corr:0.5901,mseloss:0.9397
epoch [10/20], sumloss:0.9217,mseloss:0.9178
test_epoch [10/20], loss:0.9438,corr:

In [92]:
torch.save(model.state_dict(),"/mnt/cifs/result/simple_stacked_training/init_weight/stackedmodel_initweightrun1.pth")

In [93]:

num_epochs = 20
#

#model = autoencoder()
mseloss = nn.MSELoss()

model = MixedApproachTiedMultiAutoEncoder(512,5,bias=True)
model = model.to("cuda")
optimizer = torch.optim.Adam(
model.parameters(), lr=0.0001, weight_decay=0)

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in test_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                      ,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in train_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                     ,metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(torch.mm(model.encoder5.weight,model.encoder512.weight).t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [1/20], sumloss:0.9521,mseloss:0.9479
test_epoch [1/20], loss:0.9435,corr:0.5906,mseloss:0.9394
epoch [2/20], sumloss:0.9424,mseloss:0.9381
test_epoch [2/20], loss:0.9406,corr:0.5826,mseloss:0.9364
epoch [3/20], sumloss:0.9398,mseloss:0.9355
test_epoch [3/20], loss:0.9396,corr:0.5772,mseloss:0.9353
epoch [4/20], sumloss:0.9384,mseloss:0.9341
test_epoch [4/20], loss:0.9393,corr:0.5688,mseloss:0.9350
epoch [5/20], sumloss:0.9376,mseloss:0.9333
test_epoch [5/20], loss:0.9391,corr:0.5661,mseloss:0.9348
epoch [6/20], sumloss:0.9369,mseloss:0.9325
test_epoch [6/20], loss:0.9389,corr:0.5644,mseloss:0.9345
epoch [7/20], sumloss:0.9363,mseloss:0.9319
test_epoch [7/20], loss:0.9390,corr:0.5626,mseloss:0.9346
epoch [8/20], sumloss:0.9358,mseloss:0.9314
test_epoch [8/20], loss:0.9389,corr:0.5606,mseloss:0.9345
epoch [9/20], sumloss:0.9353,mseloss:0.9309
test_epoch [9/20], loss:0.9389,corr:0.5591,mseloss:0.9344
epoch [10/20], sumloss:0.9349,mseloss:0.9305
test_epoch [10/20], loss:0.9389,corr:

In [94]:
torch.save(model.state_dict(),"/mnt/cifs/result/simple_stacked_training/init_weight/stackedmodel_initweightrun2.pth")

In [27]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30



for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([5]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T

                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)

                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)



                    num_epochs = 100
                    #
                    
                    model = MixedApproachTiedMultiAutoEncoder(512,5,bias=True)
                    model = model.to("cuda")
                    
                    mseloss = nn.MSELoss()

                    
                    #初期値weightをifで分岐
                    param = torch.load("/mnt/cifs/result/simple_stacked_training/init_weight/stackedmodel_initweight"+
                                       str(train_run)+".pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                                          ,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                                         ,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(torch.mm(model.encoder5.weight,model.encoder512.weight).cpu().t())
                        scheduler.step(mse_loss)
                    mse_summary.loc[sub_num,"simple_stacked_training"] = np.mean(test_mseloss_list)
"""
                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].detach().cpu().numpy()).to_csv("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/simple_stacked_training/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                    """

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/100], sumloss:0.9325,mseloss:0.9313
test_epoch [11/100], loss:0.9489,corr:0.9992,mseloss:0.9478
epoch [19/100], sumloss:0.9155,mseloss:0.9148
epoch [20/100], sumloss:0.9136,mseloss:0.9130
test_epoch [20/100], loss:0.9450,corr:0.9996,mseloss:0.9443


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/100], sumloss:0.9372,mseloss:0.9359
test_epoch [11/100], loss:0.9700,corr:0.9991,mseloss:0.9687
epoch [19/100], sumloss:0.9282,mseloss:0.9269
epoch [20/100], sumloss:0.9277,mseloss:0.9265
test_epoch [20/100], loss:0.9698,corr:0.9991,mseloss:0.9685


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


...

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/100], sumloss:0.9715,mseloss:0.9684
test_epoch [11/100], loss:0.9579,corr:0.9980,mseloss:0.9549
epoch [19/100], sumloss:0.9627,mseloss:0.9602
epoch [20/100], sumloss:0.9621,mseloss:0.9597
test_epoch [20/100], loss:0.9574,corr:0.9984,mseloss:0.9550


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/100], sumloss:0.9477,mseloss:0.9438
test_epoch [11/100], loss:0.9551,corr:0.9975,mseloss:0.9514
epoch [19/100], sumloss:0.9425,mseloss:0.9394
epoch [20/100], sumloss:0.9422,mseloss:0.9391
test_epoch [20/100], loss:0.9531,corr:0.9980,mseloss:0.9500





'\n                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,0],color=\'red\',alpha=0.7)\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,1],color=\'blue\',alpha=0.7)\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,2],color=\'black\',alpha=0.7)\n                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,3],color=\'green\',alpha=0.7)\n                    ax.vlines([50], -160, 160, linestyles=\'dashed\')\n                    ax.vlines([150], -160, 160, linestyles=\'dashed\')\n                    ax.vlines([250], -160, 160, linestyles=\'dashed\')\n                    ax.vlines([300], -160, 160, linestyles=\'dashed\')\n                    ax.set_xlim(0, 316)\n                    plt.savefig("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+\n                  

In [30]:
mse_summary.to_csv("/mnt/cifs/result/mse_summary.csv",index=False)

In [57]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30



for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([5]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1","run2"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T

                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)

                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    #
                    
                    model = MixedApproachTiedMultiAutoEncoder(512,5,bias=True)
                    model = model.to("cuda")
                    
                    mseloss = nn.MSELoss()

                    
                    #初期値weightをifで分岐
                    param = torch.load("/mnt/cifs/result/simple_stacked_training/init_weight/stackedmodel_initweight.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                                          ,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,torch.mm(model.encoder5.weight,model.encoder512.weight)
                                                         ,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(torch.mm(model.encoder5.weight,model.encoder512.weight).cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"test.png")
                    plt.close()


                    
                    
                    

                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316), train_encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)
                    ax.vlines([50], -160, 160, linestyles='dashed')
                    ax.vlines([150], -160, 160, linestyles='dashed')
                    ax.vlines([250], -160, 160, linestyles='dashed')
                    ax.vlines([300], -160, 160, linestyles='dashed')
                    ax.set_xlim(0, 316)
                    plt.savefig("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()



                    #E = torch.load("./weight_corr0.5868.pth")
                    #F = torch.load("./encode_output.pth")
                    #G = torch.load("./weight_corr0.7828.pth")


                    pd.DataFrame(weight_list[19].detach().cpu().numpy()).to_csv("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    pd.DataFrame(train_encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/simple_stacked_training/"
                                                                                    +str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    pd.DataFrame(encode_listlist[19].cpu().detach().numpy()).to_csv("/mnt/cifs/result/simple_stacked_training/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"test.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9195,mseloss:0.9189
test_epoch [11/20], loss:0.9419,corr:0.9996,mseloss:0.9413
epoch [19/20], sumloss:0.9058,mseloss:0.9054
epoch [20/20], sumloss:0.9050,mseloss:0.9046
test_epoch [20/20], loss:0.9408,corr:0.9997,mseloss:0.9404


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9250,mseloss:0.9245
test_epoch [11/20], loss:0.9595,corr:0.9997,mseloss:0.9590
epoch [19/20], sumloss:0.8929,mseloss:0.8925
epoch [20/20], sumloss:0.8917,mseloss:0.8912
test_epoch [20/20], loss:0.9612,corr:0.9997,mseloss:0.9608


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


...

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9260,mseloss:0.9252
test_epoch [11/20], loss:0.9658,corr:0.9994,mseloss:0.9649
epoch [19/20], sumloss:0.9167,mseloss:0.9157
epoch [20/20], sumloss:0.9162,mseloss:0.9151
test_epoch [20/20], loss:0.9665,corr:0.9993,mseloss:0.9655


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.8790,mseloss:0.8769
test_epoch [11/20], loss:0.9250,corr:0.9989,mseloss:0.9233
epoch [19/20], sumloss:0.8203,mseloss:0.8191
epoch [20/20], sumloss:0.8173,mseloss:0.8160
test_epoch [20/20], loss:0.9283,corr:0.9988,mseloss:0.9266


KeyboardInterrupt: 

In [70]:
weight_list[19].detach().numpy()

array([[-0.00753194, -0.00211688, -0.00577736, -0.00438728,  0.00125612],
       [-0.00294588,  0.00322083, -0.00429503, -0.00868926,  0.01522838],
       [-0.00148762,  0.00177953,  0.01578279, -0.01157858,  0.00809665],
       ...,
       [-0.00224451, -0.001618  ,  0.00663614, -0.0083924 ,  0.00677824],
       [-0.00038399, -0.00078969,  0.00613122, -0.0082199 , -0.00737496],
       [-0.0016073 , -0.00126362,  0.00559887, -0.00841572,  0.00345344]],
      dtype=float32)

In [72]:
weight_list[19].detach().numpy()[0]

array([-0.00753194, -0.00211688, -0.00577736, -0.00438728,  0.00125612],
      dtype=float32)

In [76]:
np.corrcoef(metanalysis[3].cpu().numpy(),weight_list[19].detach().numpy()[:,3])

array([[1.        , 0.99973952],
       [0.99973952, 1.        ]])

In [ ]:
group1_sub_path_run1 = group1_sub_path[group1_sub_path.str.contains("run1")] #run1のファイル名パス
group1_sub_path_run2 = group1_sub_path[group1_sub_path.str.contains("run2")]

sample = pd.read_csv(DATA_PATH+group1_sub_path_run1.iloc[0],sep='\t',header=None)
sample = sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(sample)
sample = scaler.transform(sample)
sample = sample[None,:,:]
for subnum in tqdm_notebook(np.sort(group1_sub_path_run1.values[1:40])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    sample = np.append(sample,A,axis=0)


test_sample = pd.read_csv(DATA_PATH+group1_sub_path_run2.iloc[0],sep='\t',header=None)
test_sample = test_sample.values[:,:-1].T
scaler = StandardScaler()
scaler.fit(test_sample)
test_sample = scaler.transform(test_sample)
test_sample = test_sample[None,:,:]

for subnum in tqdm_notebook(np.sort(group1_sub_path_run2.values[1:80])):
    
    A = pd.read_csv(DATA_PATH+subnum,sep='\t',header=None)
    A = A.values[:,:-1].T
    scaler = StandardScaler()
    scaler.fit(A)
    A = scaler.transform(A)
    A = A[None,:,:]
    test_sample = np.append(test_sample,A,axis=0)
sample = np.reshape(sample,(sample.shape[0]*316,-1))
test_sample = np.reshape(test_sample,(test_sample.shape[0]*316,-1))

In [124]:

train_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)


test_dataset = MyDataset(torch.tensor(sample,dtype=torch.float32))
test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)

In [120]:

num_epochs = 20

model = MixedApproachTiedAutoEncoder(59412,5,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [1/20], sumloss:0.9458,mseloss:0.9408
test_epoch [1/20], loss:0.9421,corr:0.5323,mseloss:0.9374
epoch [2/20], sumloss:0.9477,mseloss:0.9424
test_epoch [2/20], loss:0.9423,corr:0.5300,mseloss:0.9376
epoch [3/20], sumloss:0.9480,mseloss:0.9426
test_epoch [3/20], loss:0.9496,corr:0.4477,mseloss:0.9441
epoch [4/20], sumloss:0.9499,mseloss:0.9448
test_epoch [4/20], loss:0.9405,corr:0.5265,mseloss:0.9357
epoch [5/20], sumloss:0.9465,mseloss:0.9412
test_epoch [5/20], loss:0.9527,corr:0.4681,mseloss:0.9474
epoch [6/20], sumloss:0.9458,mseloss:0.9406
test_epoch [6/20], loss:0.9567,corr:0.5211,mseloss:0.9519
epoch [7/20], sumloss:0.9499,mseloss:0.9440
test_epoch [7/20], loss:0.9459,corr:0.4829,mseloss:0.9408
epoch [8/20], sumloss:0.9487,mseloss:0.9433
test_epoch [8/20], loss:0.9410,corr:0.4902,mseloss:0.9359
epoch [9/20], sumloss:0.9484,mseloss:0.9427
test_epoch [9/20], loss:0.9443,corr:0.5152,mseloss:0.9395
epoch [10/20], sumloss:0.9479,mseloss:0.9424
test_epoch [10/20], loss:0.9447,corr:

In [125]:

num_epochs = 20

model = MixedApproachTiedAutoEncoder(59412,5,bias=True)
#model = autoencoder()
mseloss = nn.MSELoss()
optimizer = torch.optim.Adam(
model.parameters(), lr=0.001, weight_decay=0)
model = model.to("cuda")

weight_list = list()
for epoch in range(num_epochs):
    loss_list = list()
    mseloss_list = list()
    corr_list = list()

    test_loss_list = list()
    test_mseloss_list = list()
    test_corr_list = list()
    for data in train_dataloader:
        
        data = data.to("cuda")
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        loss_list.append(loss.item())
        mseloss_list.append(mse_loss.item())
        corr_list.append(corr.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
    
    
    for data in test_dataloader:
        
        data = data.to("cuda")
        model.eval()
        # ===================forward=====================
        _,output = model(data)
        loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,0.01)
        test_loss_list.append(loss.item())
        test_mseloss_list.append(mse_loss.item())
        test_corr_list.append(corr.item())
    print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
          .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
    weight_list.append(model.encoder.weight.cpu().t())

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [1/20], sumloss:0.9458,mseloss:0.9408
test_epoch [1/20], loss:0.9421,corr:0.5323,mseloss:0.9374
epoch [2/20], sumloss:0.9477,mseloss:0.9424
test_epoch [2/20], loss:0.9423,corr:0.5300,mseloss:0.9376
epoch [3/20], sumloss:0.9480,mseloss:0.9426
test_epoch [3/20], loss:0.9496,corr:0.4477,mseloss:0.9441
epoch [4/20], sumloss:0.9499,mseloss:0.9448
test_epoch [4/20], loss:0.9405,corr:0.5265,mseloss:0.9357
epoch [5/20], sumloss:0.9465,mseloss:0.9412
test_epoch [5/20], loss:0.9527,corr:0.4681,mseloss:0.9474
epoch [6/20], sumloss:0.9458,mseloss:0.9406
test_epoch [6/20], loss:0.9567,corr:0.5211,mseloss:0.9519
epoch [7/20], sumloss:0.9499,mseloss:0.9440
test_epoch [7/20], loss:0.9459,corr:0.4829,mseloss:0.9408
epoch [8/20], sumloss:0.9487,mseloss:0.9433
test_epoch [8/20], loss:0.9410,corr:0.4902,mseloss:0.9359
epoch [9/20], sumloss:0.9484,mseloss:0.9427
test_epoch [9/20], loss:0.9443,corr:0.5152,mseloss:0.9395
epoch [10/20], sumloss:0.9479,mseloss:0.9424
test_epoch [10/20], loss:0.9447,corr:

In [126]:
torch.save(model.state_dict(),"/mnt/cifs/result/Individual_Z4/init_weight/model_initweight.pth")

In [128]:
debug=False
if debug == True:
    use_subnum=1
else:
    use_subnum=30



for corr_alpha in tqdm_notebook([1.5]):
    for node_num in tqdm_notebook([5]):
        for is_bandpass in tqdm_notebook(["MSMALL"]):
            for train_run in ["run1"]:
                for sub_num in use_subject_num[:use_subnum]:
                    if is_bandpass == "MSMALL":
                        test_bandsample_100307= pd.read_csv(DATA_PATH+str(sub_num)+
                                                            "_"+is_bandpass+"_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(DATA_PATH+str(sub_num)+
                                                                 "_"+is_bandpass+"_run2.csv",sep='\t',header=None).iloc[:,:-1]
                    else:
                        test_bandsample_100307= pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                            "_BandPassed_run1.csv",sep='\t',header=None).iloc[:,:-1]
                        test_bandsample_100307run2 = pd.read_csv(BANDDATA_PATH+str(sub_num)+
                                                                 "_BandPassed_run2.csv",sep='\t',header=None).iloc[:,:-1]

                    test_bandsample_100307 = test_bandsample_100307.T
                    test_bandsample_100307run2 = test_bandsample_100307run2.T

                    scaler = StandardScaler()
                    scaler.fit(test_bandsample_100307)
                    test_bandsample_100307 = scaler.transform(test_bandsample_100307)
                    
                    scaler2 = StandardScaler()
                    scaler2.fit(test_bandsample_100307run2)
                    test_bandsample_100307run2 = scaler2.transform(test_bandsample_100307run2)
                    
                    test_bandsample_100307 = np.append(test_bandsample_100307,test_bandsample_100307run2,
                                                      axis=0)
                    if train_run == "run1":
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=30,shuffle=True)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=30,shuffle=False)
                    else:
                        train_dataset = MyDataset(torch.tensor(test_bandsample_100307run2,dtype=torch.float))
                        train_dataloader = DataLoader(train_dataset,batch_size=20,shuffle=False)

                        test_dataset = MyDataset(torch.tensor(test_bandsample_100307,dtype=torch.float))
                        test_dataloader = DataLoader(test_dataset,batch_size=20,shuffle=False)



                    num_epochs = 20
                    model = MixedApproachTiedAutoEncoder(59412,node_num,bias= True)
                    #model = autoencoder()
                    mseloss = nn.MSELoss()
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)

                    #weightdict = model.state_dict()
                    #weightdict['encoder.weight']= torch.load("./bias/weight_corr0.4021.pth").t()
                    #初期値を作る
                    
                    #初期値weightをifで分岐
                    param = torch.load("/mnt/cifs/result/Individual_Z4/init_weight/model_initweight.pth")
                    #model.encoder.weight = nn.Parameter(torch.load("./weight_corr0.7828.pth").t())
                    model.load_state_dict(param)
                    model = model.to("cuda")
                    optimizer = torch.optim.Adam(
                    model.parameters(), lr=0.0001, weight_decay=0)
                    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3,min_lr=0)
                    weight_list = list()

                    encode_listlist = list()
                    train_encode_listlist = list()
                    for epoch in range(num_epochs):
                        loss_list = list()
                        mseloss_list = list()
                        corr_list = list()

                        test_loss_list = list()
                        test_mseloss_list = list()
                        test_corr_list = list()

                        encode_list = list()
                        train_encode_list = list()
                        for data in train_dataloader:

                            data = data.to("cuda")
                            # ===================forward=====================
                            encode,output = model(data)
                            train_encode_list.append(encode)
                            loss,corr,mse_loss = loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            loss_list.append(loss.item())
                            mseloss_list.append(mse_loss.item())
                            corr_list.append(corr.item())
                            # ===================backward====================
                            optimizer.zero_grad()
                            loss.backward()
                            optimizer.step()
                        # ===================log========================
                        if epoch in [10,18,19]:
                            print('epoch [{}/{}], sumloss:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(loss_list),np.mean(mseloss_list)))
                        train_encode_listlist.append(torch.cat(train_encode_list))

                        
                        
                        
                        
                        
                        


                        #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")
                        for data in test_dataloader:

                            data = data.to("cuda")
                            model.eval()
                            # ===================forward=====================
                            encode,output = model(data)
                            encode_list.append(encode)
                            loss,corr,mse_loss= loss_fuc(data,output,model.encoder.weight,metanalysis,corr_alpha)
                            test_loss_list.append(loss.item())
                            test_mseloss_list.append(mse_loss.item())
                            test_corr_list.append(corr.item())
                        encode_listlist.append(torch.cat(encode_list))
                        if epoch in [10,19]:
                            print('test_epoch [{}/{}], loss:{:.4f},corr:{:.4f},mseloss:{:.4f}'
                                  .format(epoch + 1, num_epochs, np.mean(test_loss_list),np.mean(test_corr_list),np.mean(test_mseloss_list)))
                        weight_list.append(model.encoder.weight.cpu().t())
                        scheduler.step(mse_loss)


                    fig, ax = plt.subplots(figsize=(10, 10), facecolor="w")
                    ax.plot(np.arange(316*2), encode_listlist[19].detach().cpu().numpy()[:,0],color='red',alpha=0.7)
                    ax.plot(np.arange(316*2), encode_listlist[19].detach().cpu().numpy()[:,1],color='blue',alpha=0.7)
                    ax.plot(np.arange(316*2), encode_listlist[19].detach().cpu().numpy()[:,2],color='black',alpha=0.7)
                    ax.plot(np.arange(316*2), encode_listlist[19].detach().cpu().numpy()[:,3],color='green',alpha=0.7)

                    ax.set_xlim(0, 316*2)
                    plt.savefig("/mnt/cifs/result/Individual_Z4/"+str(train_run)+"/"+
                                str(sub_num)+"_corr"+str(np.round(corr.item(),4))+
                                "_node"+str(node_num)+"train.png")
                    plt.close()





                    pd.DataFrame(weight_list[19].detach().cpu().numpy()).to_csv("/mnt/cifs/result/Individual_Z4/"+str(train_run)+"/"
                                                            +str(sub_num)+"weight_corr"+str(np.round(corr.item(),4))+".csv")
                    
                    pd.DataFrame(encode_listlist[19].detach().cpu().numpy()).to_csv("/mnt/cifs/result/Individual_Z4/"+str(train_run)+"/"+
                                                            str(sub_num)+"_encode_output_corr"+str(np.round(corr.item(),4))+"train.csv")
                    #pd.DataFrame(E.detach().numpy()).to_csv("./encode_output.csv")

/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9399,mseloss:0.9390
test_epoch [11/20], loss:0.9544,corr:0.9996,mseloss:0.9538
epoch [19/20], sumloss:0.9372,mseloss:0.9368
epoch [20/20], sumloss:0.9382,mseloss:0.9378
test_epoch [20/20], loss:0.9534,corr:0.9997,mseloss:0.9530


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9299,mseloss:0.9292
test_epoch [11/20], loss:0.9320,corr:0.9996,mseloss:0.9315
epoch [19/20], sumloss:0.9302,mseloss:0.9298
epoch [20/20], sumloss:0.9328,mseloss:0.9323
test_epoch [20/20], loss:0.9317,corr:0.9997,mseloss:0.9313


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


...

epoch [11/20], sumloss:0.9083,mseloss:0.9066
test_epoch [11/20], loss:0.9022,corr:0.9989,mseloss:0.9006
epoch [19/20], sumloss:0.8957,mseloss:0.8941
epoch [20/20], sumloss:0.8961,mseloss:0.8947
test_epoch [20/20], loss:0.8929,corr:0.9990,mseloss:0.8914


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9313,mseloss:0.9303
test_epoch [11/20], loss:0.9298,corr:0.9992,mseloss:0.9286
epoch [19/20], sumloss:0.9284,mseloss:0.9273
epoch [20/20], sumloss:0.9264,mseloss:0.9253
test_epoch [20/20], loss:0.9277,corr:0.9993,mseloss:0.9267


/home/chikazoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


epoch [11/20], sumloss:0.9281,mseloss:0.9273
test_epoch [11/20], loss:0.9338,corr:0.9995,mseloss:0.9331
epoch [19/20], sumloss:0.9293,mseloss:0.9286
epoch [20/20], sumloss:0.9270,mseloss:0.9263
test_epoch [20/20], loss:0.9331,corr:0.9996,mseloss:0.9325



